He utilizado el dataset "DEALS_DETAIL-DUMMY_NUMBERS.csv" para realizar un análisis exploratorios con el objetivo de obtener información valiosa y luego realizar un análisis de series la serie temporal y obtener un pronóstico de ventas de 7 días. 

In [ ]:
!pip3 install -U pmdarima
!pip3 install -U tbats
!pip3 install -U keras-tcn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Librerias

In [ ]:
import gc
import sys
import glob
import re
import os
import pickle
import urllib
import json
import datetime
import holidays
import pprint
import warnings
import time
import itertools
from tqdm import tqdm_notebook
from math import sqrt
import numpy as np
from numpy import savetxt
import pandas as pd
import seaborn as sns
from collections import defaultdict
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import InputLayer, Layer, Dense, LSTM, GRU, Flatten
from tensorflow.keras.layers import Conv1D, MaxPool1D, Dropout, Activation, Masking
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from keras.utils.vis_utils import plot_model
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss, ARMA
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults
from statsmodels.tsa.statespace.sarimax import SARIMAX, SARIMAXResults
from statsmodels.graphics import tsaplots as sgt
from sklearn.metrics import mean_absolute_error, mean_squared_error, max_error
from sklearn.metrics import r2_score, explained_variance_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tcn import TCN
from pmdarima import auto_arima
from tbats import BATS, TBATS

%matplotlib inline
plt.style.use('fast')
plt.rcParams["figure.figsize"] = (15, 7)
sns.set_context("paper", rc={"lines.linewidth": 1})
tf.random.set_seed(1)
tf.get_logger().setLevel('ERROR')
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)
pd.set_option("display.max_rows", 100)
warnings.filterwarnings("ignore")
np.set_printoptions(threshold=sys.maxsize)
sns.set(style='whitegrid', palette='muted')
%config InlineBackend.figure_format = 'retina'

# Diccionarios para guardar los resultados
res_pronos = defaultdict(lambda: {})
res_tiempos = defaultdict(lambda: {})

___

# Funciones

### Funciones Auxiliares

In [ ]:
def plot_line(df, ttl, path):
    plt.plot(df)
    plt.title(ttl, fontsize=12)
    plt.yticks(fontsize=10)
    plt.xticks(pd.date_range(df.index.min(),
                             df.index.max(),
                             freq='7D'),
               rotation='vertical',
               size=5)
    plt.xlabel(df.index.name, fontsize=10)
    plt.ylabel(df.columns[0], fontsize=10)
    plt.savefig(os.path.join(path, df.columns[0]+"-"+df.index.name+".pdf"))
    plt.show()

def fit_model(x_tr, y_tr, b_s):
    print("\n{}: Inicio entrenamiento".format(datetime.datetime.now()))
    history = model.fit(x_tr, y_tr, epochs=500, batch_size=b_s, verbose=1, 
                        validation_split=.3, callbacks=mis_callbacks)
    return history

def plot_pronos(data, nombre, column):
    print("\n{}: Plot pronóstico modelo:\t{}".format(datetime.datetime.now(), nombre))
    prono_plot_path = os.path.join(plot_mercado_path, nombre + "_p.png")

    '''Plot de los pronósticos'''
    data.filter(items=[column, nombre], axis=1).plot(figsize=(15, 7))
    plt.ylabel(column)
    plt.xlabel('Fecha')
    plt.title(column.replace("_", " ") +
              "\nPronósticos de %i hs." % len(data) +
              "\n Modelo %s" % nombre.rstrip("-" + column).replace("_", " "),
              size=24)
    # plt.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    plt.autoscale()
    plt.savefig(prono_plot_path, bbox_inches="tight")
    plt.show()

def forecast_nn():
    print("{}: Pronosticando".format(datetime.datetime.now()))
    fc = model.predict(x_test, verbose=1)
    fc = scaler.inverse_transform(fc)[0]
    df_test[nombre_modelo] = fc
    pronos_tabla_path = os.path.join(tabla_mercado_path, nombre_modelo + "_p.csv")
    print("\n{}: Guardando pronósticos en:\t{}".format(datetime.datetime.now(), pronos_tabla_path))
    df_test.to_csv(pronos_tabla_path)
    return fc

def forecast(modelo):    
    print("{}: Pronosticando".format(datetime.datetime.now()))
    fc = modelo.forecast(horizonte_prevision, alpha=0.05)[0]
    df_test[nombre_modelo] = fc
    pronos_tabla_path = os.path.join(tabla_mercado_path, nombre_modelo + "_p.csv")
    print("\n{}: Guardando pronósticos en:\t{}".format(datetime.datetime.now(), pronos_tabla_path))
    df_test.to_csv(pronos_tabla_path)
    return fc

def prediction(modelo):
    print("\n{}: Pronosticando".format(datetime.datetime.now()))
    fc = modelo.predict(horizonte_prevision)
    df_test[nombre_modelo] = fc
    pronos_tabla_path = os.path.join(tabla_mercado_path, nombre_modelo + "_p.csv")
    print("\n{}: Guardando pronósticos en:\t{}".format(datetime.datetime.now(), pronos_tabla_path))
    df_test.to_csv(pronos_tabla_path)
    return fc

def diagnostic(modelo):    
    print("\n{}: plot_diagnostics: ".format(datetime.datetime.now()))
    diagnostics_plot_path = os.path.join(plot_mercado_path,
                                            nombre_modelo + "_d.png")
    modelo.plot_diagnostics(figsize=(15, 15))
    plt.savefig(diagnostics_plot_path)
    plt.show()

def plot_pronos_sarima(modelo, test):
    pred = modelo.get_prediction(start=df_test.index.min(), 
                                                    end=df_test.index.max(), 
                                                    full_results=True)
    pred_val = pred.conf_int()
    ax = test.plot() 
    pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', 
                               alpha=.7, figsize=(14, 7))
    ax.fill_between(pred_val.index,
                    pred_val.iloc[:, 0],
                    pred_val.iloc[:, 1], color='k', alpha=.2)
    ax.set_xlabel('Date')
    ax.set_ylabel(test.columns[0])
    plt.legend()
    plt.show()


def plot_pie_cant(x, y):
    total_cant_dia = y.sum()
    total_cant_dia = str(int(total_cant_dia))

    plt.rcParams['font.size'] = 10.0
    plt.rcParams['font.weight'] = 6

    def autopct_format(values):
        def my_format(pct):
            total = sum(values)
            val = int(round(pct * total / 100.0))
            return ' {v:d}'.format(v=val)

        return my_format

    colors = ['#BC243C', '#FE840E', '#C62168']
    explode = explode = tuple([0.05] * len(x))
    fig1, ax1 = plt.subplots()

    ax1.pie(y,
            colors=colors,
            labels=x,
            autopct=autopct_format(y),
            startangle=90,
            explode=explode)
    centre_circle = plt.Circle((0, 0), 0.82, fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)

    ax1.axis('equal')

    label = ax1.annotate('Total \n' + str(total_cant_dia),
                         color='red',
                         xy=(0, 0),
                         fontsize=10,
                         ha="center")
    plt.tight_layout()
    plt.show()


def plot_pie_comi(x, y):
    total_cant_dia = y.sum()
    total_cant_dia = str(int(total_cant_dia))
    plt.rcParams["figure.figsize"] = (13, 5)
    plt.rcParams['font.size'] = 10
    plt.rcParams['font.weight'] = 6

    def autopct_format(values):
        def my_format(pct):
            total = sum(values)
            val = int(round(pct * total / 100.0))
            return '${v:d}'.format(v=val)

        return my_format

    colors = ['#BC243C', '#FE840E', '#C62168']
    explode = explode = tuple([0.05] * len(x))
    fig1, ax1 = plt.subplots()

    ax1.pie(y,
            colors=colors,
            labels=x,
            autopct=autopct_format(y),
            startangle=90,
            explode=explode)
    centre_circle = plt.Circle((0, 0), 0.82, fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)

    ax1.axis('equal')

    label = ax1.annotate('Total \n' + '$' + str(total_cant_dia),
                         color='red',
                         xy=(0, 0),
                         fontsize=10,
                         ha="center")
    plt.tight_layout()
    plt.show()

def plot_bar(x, y, title, xlab, ylab):
    plt.title(title, fontsize=18)
    plt.bar(x, y, color='#99ff99', edgecolor='green', linewidth=1)
    plt.xlabel(xlab, fontsize=15)
    plt.ylabel(ylab, fontsize=15)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    for k, v in y.items():
        plt.text(k,
                 round(v),
                 str(round(v)),
                 fontsize=12,
                 color='k',
                 horizontalalignment='center',
                 verticalalignment='bottom')
    plt.show()

def plot_metrics(res_pronos_df, plot_mercado_path, RE):
    res_pronos_df = res_pronos_df.filter(regex=RE, axis=0)
    res_pronos_df.sort_values(by=['MAE'], ascending=False).plot.barh(
        y='MAE', legend=None, figsize=(15, len(res_pronos_df)))
    plt.title(column.replace("_", " ") + '\nError absoluto medio', size=18)
    mae_plot_path = os.path.join(plot_mercado_path, "MAE.png")
    plt.savefig(mae_plot_path)
    plt.show()
    res_pronos_df.sort_values(by=['MSE'], ascending=False).plot.barh(
        y='MSE', legend=None, figsize=(15, len(res_pronos_df)))
    plt.title(column.replace("_", " ") + '\nError cuadrático medio', size=18)
    mse_plot_path = os.path.join(plot_mercado_path, "MSE.png")
    plt.savefig(mse_plot_path)
    plt.show()
    res_pronos_df.sort_values(by=['R2'], ascending=True).plot.barh(
        y='R2', legend=None, figsize=(15, len(res_pronos_df)))
    plt.title(column.replace("_", " ") + '\nCoeficiente de determinación',
              size=18)
    r2_plot_path = os.path.join(plot_mercado_path, "R2.png")
    plt.savefig(r2_plot_path)
    plt.show()
    res_pronos_df.sort_values(by=['EVS'], ascending=True).plot.barh(
        y='EVS', legend=None, figsize=(15, len(res_pronos_df)))
    plt.title(column.replace("_", " ") + '\nVarianza explicada', size=18)
    evs_plot_path = os.path.join(plot_mercado_path, "EVS.png")
    plt.savefig(evs_plot_path)
    plt.show()
    res_pronos_df.sort_values(by=['ME'], ascending=False).plot.barh(
        y='ME', legend=None, figsize=(15, len(res_pronos_df)))
    plt.title(column.replace("_", " ") + '\nError máximo', size=18)
    me_plot_path = os.path.join(plot_mercado_path, "ME.png")
    plt.savefig(me_plot_path)
    plt.show()
    res_pronos_df.sort_values(by=['MAPE'], ascending=False).plot.barh(
        y='MAPE', legend=None, figsize=(15, len(res_pronos_df)))
    plt.title(column.replace("_", " ") + '\nError Porcentual Absoluto Medio',
              size=18)
    mape_plot_path = os.path.join(plot_mercado_path, "MAPE.png")
    plt.savefig(mape_plot_path)
    plt.show()


def tiempo_entrenamiento():
    t_total = datetime.datetime.now() - tiempo_inicio_fit
    print("\n{}: Tiempo total:\t{}".format(datetime.datetime.now(),t_total))
    res_tiempos['t_total'][nombre_modelo] = t_total.total_seconds()
    time_tabla_path = os.path.join(tabla_mercado_path,nombre_modelo + "_t.csv")
    res_tiempos_df = pd.DataFrame(res_tiempos)
    res_tiempos_df.to_csv(time_tabla_path)
    print("\n{}: Guardando tiempo total en:\t{}\n\n".format(datetime.datetime.now(), time_tabla_path))

def mk_model_dirs(name):
    '''Crea las carpetas y los paths hacia las carpetas de los modelos'''
    modelo_path = os.path.join(modelos_path, name)
    tabla_path = os.path.join(tablas_path, name)
    plot_path = os.path.join(plots_path, name)
    # log_path = os.path.join(logs_path, name)
    for model_dirs in [modelo_path, tabla_path, plot_path]:
        if not os.path.exists(model_dirs):
            try:
                os.makedirs(model_dirs, 0o700)
            except OSError as e:
                if e.errno != errno.EEXIST:
                    raise
            print("Directorio creado: {}".format(model_dirs))
        else:
            print("Directorio ya existente: {}".format(model_dirs))
    return modelo_path, tabla_path, plot_path


def mk_model_mercado_dirs(mercado):
    modelo_mercado_path = os.path.join(modelo_path, mercado)
    tabla_mercado_path = os.path.join(tabla_path, mercado)
    plot_mercado_path = os.path.join(plot_path, mercado)
    # log_mercado_path = os.path.join(log_path, mercado)
    for model_mercado_dirs in [
            modelo_mercado_path, tabla_mercado_path, plot_mercado_path
    ]:
        if not os.path.exists(model_mercado_dirs):
            try:
                os.makedirs(model_mercado_dirs, 0o700)
            except OSError as e:
                if e.errno != errno.EEXIST:
                    raise
            print("Directorio creado: {}".format(model_mercado_dirs))
        else:
            print("Directorio ya existente: {}".format(model_mercado_dirs))
    return modelo_mercado_path, tabla_mercado_path, plot_mercado_path



def mk_dir(name):
    '''crear el directorio con el nombre que le pasas'''
    path = os.path.join(os.getcwd(), name)
    if not os.path.exists(path):
        try:
            os.makedirs(path, 0o700)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise
        print("Directorio creado: {}".format(path))
    else:
        print("Directorio ya existente: {}".format(path))
    return path

def traducir_dia(text):
    t = text.replace('Sunday', 'Domingo')
    t = t.replace('Monday', 'Lunes')
    t = t.replace('Tuesday', 'Martes')
    t = t.replace('Wednesday', 'Miércoles')
    t = t.replace('Thursday', 'Jueves')
    t = t.replace('Friday', 'Viernes')
    t = t.replace('Saturday', 'Sábado')
    return t


def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def metrics_time_series(y_true, y_pred, resultados, modelo):
    print("\n{}: Evaluando pronósticos".format(datetime.datetime.now()))
    me = round(max_error(y_true, y_pred), 3)
    r2 = round(r2_score(y_true, y_pred), 3)
    evs = round(explained_variance_score(y_true, y_pred), 3)
    mae = round(mean_absolute_error(y_true, y_pred), 3)
    mse = round(mean_squared_error(y_true, y_pred), 3)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 3)
    print(f"Error máximo: {me}")
    print(f"Coeficiente de determinación (R2): {r2}")
    print(f"Varianza explicada: {evs}")
    print(f"Error cuadrático medio: {mse}")
    print(f"Error absoluto medio: {mae}")
    print(f"Error Porcentual Absoluto Medio: {mape}")
    resultados['ME'][modelo] = me
    resultados['R2'][modelo] = r2
    resultados['EVS'][modelo] = evs
    resultados['MAE'][modelo] = mae
    resultados['MSE'][modelo] = mse
    resultados['MAPE'][modelo] = mape

    metric_tabla_path = os.path.join(tabla_mercado_path, modelo + "_m.csv")
    res_pronos_df = pd.DataFrame(resultados)
    print("\n{}: Guardando métricas en:\t{}".format(datetime.datetime.now(),metric_tabla_path))
    res_pronos_df.filter(regex=RE, axis=0).to_csv(metric_tabla_path)


def datos_supervisados(entrenamiento_esc, test_esc, pasado_historico):
    x_entrenamiento, y_entrenamiento = [], []
    for j in range(0, entrenamiento_esc.shape[0] - len(test_esc) + 1):
        indices = range(j - pasado_historico, j, 1)
        x_entrenamiento.append(
            np.reshape(entrenamiento_esc[indices], (pasado_historico, 1)))
        y_entrenamiento.append(entrenamiento_esc[j:j + len(test_esc)])
    x_train = np.asarray(x_entrenamiento)
    y_train = np.asarray(y_entrenamiento)
    x_test = np.reshape(entrenamiento_esc[-pasado_historico:],
                        (1, pasado_historico, 1))
    y_test = np.reshape(test_esc, (1, test_esc.shape[0]))
    return x_train, y_train, x_test, y_test

def plot_history(history):
    history_plot_path = os.path.join(plot_mercado_path, nombre_modelo + "-history.png")
    plt.figure()
    plt.xlabel('Épocas')
    plt.ylabel('Error absoluto medio')
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.legend(['Entrenamiento', 'Validación'])
    plt.title(nombre_modelo, size=24)
    plt.savefig(history_plot_path)
    plt.show()

def mk_regexs(mercado):
    regex_AUTOARIMA = re.compile(r'^' + mercado + '|^AUTOARIMA.+' +
                                 mercado.replace("Precio_mercado_SPOT_", "") +
                                 "$")
    regex_AR = re.compile(r'^' + mercado + '|^AR_p.+' +
                          mercado.replace("Precio_mercado_SPOT_", "") + "$")
    regex_MA = re.compile(r'^' + mercado + '|^MA_q.+' +
                          mercado.replace("Precio_mercado_SPOT_", "") + "$")
    regex_ARMA = re.compile(r'^' + mercado + '|^ARMA.+' +
                            mercado.replace("Precio_mercado_SPOT_", "") + "$")
    regex_ARIMA = re.compile(r'^' + mercado + '|^ARIMA.+' +
                             mercado.replace("Precio_mercado_SPOT_", "") + "$")
    regex_SARIMA = re.compile(r'^' + mercado + '|^SARIMA.+' +
                              mercado.replace("Precio_mercado_SPOT_", "") +
                              "$")
    regex_BATS = re.compile(r'^' + mercado + '|^BATS.+' +
                            mercado.replace("Precio_mercado_SPOT_", "") + "$")
    regex_TBATS = re.compile(r'^' + mercado + '|^TBATS.+' +
                             mercado.replace("Precio_mercado_SPOT_", "") + "$")
    regex_lstm_densa = re.compile(r'^' + mercado + '|^LSTM\d+?_Densa\d+?.+' +
                                  mercado.replace('Precio_mercado_SPOT_', '') +
                                  '$')
    regex_lstm_densa_do = re.compile(
        r'^' + mercado + '|^LSTM\d+?_Densa\d+?_DO\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_lstm_lstm_densa = re.compile(
        r'^' + mercado + '|^LSTM\d+?_LSTM\d+?_Densa\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_lstm_lstm_densa_do = re.compile(
        r'^' + mercado + '|^LSTM\d+?_LSTM\d+?_Densa\d+?_DO\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_lstm_lstm_densa_densa = re.compile(
        r'^' + mercado + '|^LSTM\d+?_LSTM\d+?_Densa\d+?_Densa\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_lstm_lstm_densa_densa_do = re.compile(
        r'^' + mercado + '|^LSTM\d+?_LSTM\d+?_Densa\d+?_Densa\d+?_DO\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_lstm_lstm_lstm_densa = re.compile(
        r'^' + mercado + '|^LSTM\d+?_LSTM\d+?_LSTM\d+?_Densa\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_lstm_lstm_lstm_densa_do = re.compile(
        r'^' + mercado + '|^LSTM\d+?_LSTM\d+?_LSTM\d+?_Densa\d+?_DO\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_gru_flatten_densa = re.compile(
        r'^' + mercado + '|^GRU\d+?_Flatten_Densa\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_gru_do_flatten_densa = re.compile(
        r'^' + mercado + '|^GRU\d+?_DO\d+?_Flatten_Densa\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_gru_gru_flatten_densa = re.compile(
        r'^' + mercado + '|^GRU\d+?_GRU\d+?_Flatten_Densa\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_gru_do_gru_do_flatten_densa = re.compile(
        r'^' + mercado +
        '|^GRU\d+?_DO\d+?_GRU\d+?_DO\d+?_Flatten_Densa\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_gru_gru_flatten_densa_densa = re.compile(
        r'^' + mercado + '|^GRU\d+?_GRU\d+?_Flatten_Densa\d+?_Densa\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_gru_do_gru_do_flatten_densa_densa = re.compile(
        r'^' + mercado +
        '|^GRU\d+?_DO\d+?_GRU\d+?_DO\d+?_Flatten_Densa\d+?_Densa\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_gru_gru_gru_flatten_densa = re.compile(
        r'^' + mercado + '|^GRU\d+?_GRU\d+?_GRU\d+?_Flatten_Densa\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_gru_do_gru_do_gru_do_flatten_densa = re.compile(
        r'^' + mercado +
        '|^GRU\d+?_DO\d+?_GRU\d+?_DO\d+?_GRU\d+?_DO\d+?_Flatten_Densa\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_conv1d_maxpool1d_flatten_densa = re.compile(
        r'^' + mercado + '|^Conv1D\d+?_MaxPool1D\d+?_Flatten_Densa\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_conv1d_do_maxpool1d_flatten_densa = re.compile(
        r'^' + mercado +
        '|^Conv1D\d+?_DO\d+?_MaxPool1D\d+?_Flatten_Densa\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_conv1d_maxpool1d_conv1d_maxpool1d_flatten_densa = re.compile(
        r'^' + mercado +
        '|^Conv1D\d+?_MaxPool1D\d+?_Conv1D\d+?_MaxPool1D\d+?_Flatten_Densa\d+?.+'
        + mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_conv1d_do_maxpool1d_conv1d_do_maxpool1d_flatten_densa = re.compile(
        r'^' + mercado +
        '|^Conv1D\d+?_DO\d+?_MaxPool1D\d+?_Conv1D\d+?_DO\d+?_MaxPool1D\d+?_Flatten_Densa\d+?.+'
        + mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_TCN_densa = re.compile(r'^' + mercado + '|^TCN\d+?_Densa\d+?.+' +
                                 mercado.replace('Precio_mercado_SPOT_', '') +
                                 '$')
    regex_TCN_do_densa = re.compile(
        r'^' + mercado + '|^TCN\d+?_DO\d+?_Densa\d+?.+' +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_lstm_densa = re.compile(
        r"^" + mercado + "|^Masking_LSTM\d+?_Densa\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_lstm_densa_do = re.compile(
        r"^" + mercado + "|^Masking_LSTM\d+?_Densa\d+?_DO\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_lstm_lstm_densa = re.compile(
        r"^" + mercado + "|^Masking_LSTM\d+?_LSTM\d+?_Densa\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_lstm_lstm_densa_do = re.compile(
        r"^" + mercado + "|^Masking_LSTM\d+?_LSTM\d+?_Densa\d+?_DO\d+?" +
        mercado.replace("Perecio_mercado_SPOT_", '') + "$")
    regex_masking_lstm_lstm_densa_densa = re.compile(
        r"^" + mercado + "|^Masking_LSTM\d+?_LSTM\d+?_Densa\d+?_Densa\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_lstm_lstm_densa_densa_do = re.compile(
        r"^" + mercado +
        "|^Masking_LSTM\d+?_LSTM\d+?_Densa\d+?_Densa\d+?_DO\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_lstm_lstm_lstm_densa = re.compile(
        r"^" + mercado + "|^Masking_LSTM\d+?_LSTM\d+?_LSTM\d+?_Densa\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_lstm_lstm_lstm_densa_do = re.compile(
        r"^" + mercado +
        "|^Masking_LSTM\d+?_LSTM\d+?_LSTM\d+?_Densa\d+?_DO\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_gru_flatten_densa = re.compile(
        r"^" + mercado + "|^Masking_GRU\d+?_Flatten_Densa\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_gru_do_flatten_densa = re.compile(
        r"^" + mercado + "|^Masking_GRU\d+?_DO\d+?_Flatten_Densa\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_gru_gru_flatten_densa = re.compile(
        r"^" + mercado + "|^Masking_GRU\d+?_GRU\d+?_Flatten_Densa\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_gru_do_gru_do_flatten_densa = re.compile(
        r"^" + mercado +
        "|^Masking_GRU\d+?_DO\d+?_GRU\d+?_DO\d+?_Flatten_Densa\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_gru_gru_flatten_densa_densa = re.compile(
        r"^" + mercado +
        "|^Masking_GRU\d+?_GRU\d+?_Flatten_Densa\d+?_Densa\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_gru_do_gru_do_flatten_densa_densa = re.compile(
        r"^" + mercado +
        "|^Masking_GRU\d+?_DO\d+?_GRU\d+?_DO\d+?_Flatten_Densa\d+?_Densa\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_gru_gru_gru_flatten_densa = re.compile(
        r"^" + mercado +
        "|^Masking_GRU\d+?_GRU\d+?_GRU\d+?_Flatten_Densa\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_gru_do_gru_do_gru_do_flatten_densa = re.compile(
        r"^" + mercado +
        "|^Masking_GRU\d+?_DO\d+?_GRU\d+?_DO\d+?_GRU\d+?_DO\d+?_Flatten_Densa\d+?"
        + mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_conv1d_maxpool1d_flatten_densa = re.compile(
        r"^" + mercado +
        "|^Masking_Conv1D\d+?_MaxPool1D\d+?_Flatten_Densa\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_conv1d_do_maxpool1d_flatten_densa = re.compile(
        r"^" + mercado +
        "|^Masking_Conv1D\d+?_DO\d+?_MaxPool1D\d+?_Flatten_Densa\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_conv1d_maxpool1d_conv1d_maxpool1d_flatten_densa = re.compile(
        r"^" + mercado +
        "|^Masking_Conv1D\d+?_MaxPool1D\d+?_Conv1D\d+?_MaxPool1D\d+?_Flatten_Densa\d+?"
        + mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_conv1d_do_maxpool1d_conv1d_do_maxpool1d_flatten_densa = re.compile(
        r"^" + mercado +
        "|^Masking_Conv1D\d+?_DO\d+?_MaxPool1D\d+?_Conv1D\d+?_DO\d+?_MaxPool1D\d+?_Flatten_Densa\d+?"
        + mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_TCN_densa = re.compile(
        r"^" + mercado + "|^Masking_TCN\d+?_Densa\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    regex_masking_TCN_do_densa = re.compile(
        r"^" + mercado + "|^Masking_TCN\d+?_DO\d+?_Densa\d+?" +
        mercado.replace('Precio_mercado_SPOT_', '') + '$')
    return regex_AUTOARIMA, regex_AR, regex_MA, regex_ARMA, regex_ARIMA, regex_SARIMA, regex_BATS, regex_TBATS, regex_lstm_densa, regex_lstm_densa_do, regex_lstm_lstm_densa, regex_lstm_lstm_densa_do, regex_lstm_lstm_densa_densa, regex_lstm_lstm_densa_densa_do, regex_lstm_lstm_lstm_densa, regex_lstm_lstm_lstm_densa_do, regex_gru_flatten_densa, regex_gru_do_flatten_densa, regex_gru_gru_flatten_densa, regex_gru_do_gru_do_flatten_densa, regex_gru_gru_flatten_densa_densa, regex_gru_do_gru_do_flatten_densa_densa, regex_gru_gru_gru_flatten_densa, regex_gru_do_gru_do_gru_do_flatten_densa, regex_conv1d_maxpool1d_flatten_densa, regex_conv1d_do_maxpool1d_flatten_densa, regex_conv1d_maxpool1d_conv1d_maxpool1d_flatten_densa, regex_conv1d_do_maxpool1d_conv1d_do_maxpool1d_flatten_densa, regex_TCN_densa, regex_TCN_do_densa, regex_masking_lstm_densa, regex_masking_lstm_densa_do, regex_masking_lstm_lstm_densa, regex_masking_lstm_lstm_densa_do, regex_masking_lstm_lstm_densa_densa, regex_masking_lstm_lstm_densa_densa_do, regex_masking_lstm_lstm_lstm_densa, regex_masking_lstm_lstm_lstm_densa_do, regex_masking_gru_flatten_densa, regex_masking_gru_do_flatten_densa, regex_masking_gru_gru_flatten_densa, regex_masking_gru_do_gru_do_flatten_densa, regex_masking_gru_gru_flatten_densa_densa, regex_masking_gru_do_gru_do_flatten_densa_densa, regex_masking_gru_gru_gru_flatten_densa, regex_masking_gru_do_gru_do_gru_do_flatten_densa, regex_masking_conv1d_maxpool1d_flatten_densa, regex_masking_conv1d_do_maxpool1d_flatten_densa, regex_masking_conv1d_maxpool1d_conv1d_maxpool1d_flatten_densa, regex_masking_conv1d_do_maxpool1d_conv1d_do_maxpool1d_flatten_densa, regex_masking_TCN_densa, regex_masking_TCN_do_densa

### Funciones: Redes Neuronales

In [ ]:
# 08- LSTM Densa
# crear_lstm_densa
def crear_lstm_densa(u_0,
                     pasado_historico,
                     horizonte_prevision,
                     n_artributos=1):
    """crear_lstm_densa"""
    model = Sequential(
        name="LSTM{}_Densa{}".format(u_0, horizonte_prevision))
    model.add(
        LSTM(units=u_0, input_shape=(pasado_historico, n_artributos)))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model

# crear_masking_lstm_densa
def crear_masking_lstm_densa(u_0,
                             pasado_historico,
                             horizonte_prevision,
                             n_artributos=1):
    """crear_masking_lstm_densa"""
    model = Sequential(
        name="Masking_LSTM{}_Densa{}".format(u_0, horizonte_prevision))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(LSTM(units=u_0))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model

# 09- LSTM Densa DO
# crear_lstm_densa_do
def crear_lstm_densa_do(u_0,
                        pasado_historico,
                        horizonte_prevision,
                        dropout=0.1,
                        n_artributos=1):
    """crear_lstm_densa_do"""
    model = Sequential(name="LSTM{}_Densa{}_DO{}".format(
        u_0, horizonte_prevision, int(dropout * 100)))
    model.add(
        LSTM(units=u_0,
             dropout=dropout,
             input_shape=(pasado_historico, n_artributos)))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_lstm_densa_do
def crear_masking_lstm_densa_do(u_0,
                                pasado_historico,
                                horizonte_prevision,
                                dropout=0.1,
                                n_artributos=1):
    """crear_masking_lstm_densa_do"""
    model = Sequential(name="Masking_LSTM{}_Densa{}_DO{}".format(
        u_0, horizonte_prevision, int(dropout * 100)))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(LSTM(units=u_0, dropout=dropout))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 10- LSTM LSTM Densa
# crear_lstm_lstm_densa
def crear_lstm_lstm_densa(u_1,
                          u_2,
                          pasado_historico,
                          horizonte_prevision,
                          n_artributos=1):
    """crear_lstm_lstm_densa"""
    model = Sequential(name="LSTM{}_LSTM{}_Densa{}".format(
        u_1, u_2, horizonte_prevision))
    model.add(
        LSTM(units=u_1,
             return_sequences=True,
             input_shape=(pasado_historico, n_artributos)))
    model.add(LSTM(units=u_2))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_lstm_lstm_densa
def crear_masking_lstm_lstm_densa(u_1,
                                  u_2,
                                  pasado_historico,
                                  horizonte_prevision,
                                  n_artributos=1):
    model = Sequential(name="Masking_LSTM{}_LSTM{}_Densa{}".format(
        u_1, u_2, horizonte_prevision))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(LSTM(units=u_1, return_sequences=True))
    model.add(LSTM(units=u_2))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 11- LSTM LSTM Densa_DO
# crear_lstm_lstm_densa_do
def crear_lstm_lstm_densa_do(u_1,
                             u_2,
                             pasado_historico,
                             horizonte_prevision,
                             dropout=0.1,
                             n_artributos=1):
    model = Sequential(name="LSTM{}_LSTM{}_Densa{}_DO{}".format(
        u_1, u_2, horizonte_prevision, int(dropout * 100)))
    model.add(
        LSTM(units=u_1,
             dropout=dropout,
             return_sequences=True,
             input_shape=(pasado_historico, n_artributos)))
    model.add(LSTM(units=u_2, dropout=dropout))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_lstm_lstm_densa_do
def crear_masking_lstm_lstm_densa_do(u_1,
                                     u_2,
                                     pasado_historico,
                                     horizonte_prevision,
                                     dropout=0.1,
                                     n_artributos=1):
    model = Sequential(name="Masking_LSTM{}_LSTM{}_Densa{}_DO{}".format(
        u_1, u_2, horizonte_prevision, int(dropout * 100)))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(LSTM(units=u_1, dropout=dropout, return_sequences=True))
    model.add(LSTM(units=u_2, dropout=dropout))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 12- LSTM LSTM Densa Densa
# crear_lstm_lstm_densa_densa
def crear_lstm_lstm_densa_densa(u_1,
                                u_2,
                                pasado_historico,
                                horizonte_prevision,
                                n_artributos=1):
    model = Sequential(name="LSTM{}_LSTM{}_Densa{}_Densa{}".format(
        u_1, u_2, u_1, horizonte_prevision))
    model.add(
        LSTM(units=u_1,
             return_sequences=True,
             input_shape=(pasado_historico, n_artributos)))
    model.add(LSTM(units=u_2))
    model.add(Dense(units=u_1))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_lstm_lstm_densa_densa
def crear_masking_lstm_lstm_densa_densa(u_1,
                                        u_2,
                                        pasado_historico,
                                        horizonte_prevision,
                                        n_artributos=1):
    model = Sequential(name="Masking_LSTM{}_LSTM{}_Densa{}_Densa{}".format(
        u_1, u_2, u_1, horizonte_prevision))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(LSTM(units=u_1, return_sequences=True))
    model.add(LSTM(units=u_2))
    model.add(Dense(units=u_1))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 13- LSTM LSTM Densa Densa_DO
# crear_lstm_lstm_densa_densa_do
def crear_lstm_lstm_densa_densa_do(u_1,
                                   u_2,
                                   pasado_historico,
                                   horizonte_prevision,
                                   dropout=0.1,
                                   n_artributos=1):
    model = Sequential(name="LSTM{}_LSTM{}_Densa{}_Densa{}_DO{}".format(
        u_1, u_2, u_1, horizonte_prevision,
        int(dropout * 100)))
    model.add(
        LSTM(units=u_1,
             dropout=dropout,
             return_sequences=True,
             input_shape=(pasado_historico, n_artributos)))
    model.add(LSTM(units=u_2, dropout=dropout))
    model.add(Dense(units=u_1))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_lstm_lstm_densa_densa_do
def crear_masking_lstm_lstm_densa_densa_do(u_1,
                                           u_2,
                                           pasado_historico,
                                           horizonte_prevision,
                                           dropout=0.1,
                                           n_artributos=1):
    model = Sequential(
        name="Masking_LSTM{}_LSTM{}_Densa{}_Densa{}_DO{}".format(
            u_1, u_2, u_1, horizonte_prevision,
            int(dropout * 100)))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(LSTM(units=u_1, dropout=dropout, return_sequences=True))
    model.add(LSTM(units=u_2, dropout=dropout))
    model.add(Dense(units=u_1))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 14- LSTM LSTM LSTM_Densa
# crear_lstm_lstm_lstm_densa
def crear_lstm_lstm_lstm_densa(u_1,
                               u_2,
                               u_3,
                               pasado_historico,
                               horizonte_prevision,
                               n_artributos=1):
    model = Sequential(name="LSTM{}_LSTM{}_LSTM{}_Densa{}".format(
        u_1, u_2, u_3, horizonte_prevision))
    model.add(
        LSTM(units=u_1,
             return_sequences=True,
             input_shape=(pasado_historico, n_artributos)))
    model.add(LSTM(units=u_2, return_sequences=True))
    model.add(LSTM(units=u_3))
    model.add(Dense(units=u_1))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_lstm_lstm_lstm_densa
def crear_masking_lstm_lstm_lstm_densa(u_1,
                                       u_2,
                                       u_3,
                                       pasado_historico,
                                       horizonte_prevision,
                                       n_artributos=1):
    model = Sequential(name="Masking_LSTM{}_LSTM{}_LSTM{}_Densa{}".format(
        u_1, u_2, u_3, horizonte_prevision))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(LSTM(units=u_1, return_sequences=True))
    model.add(LSTM(units=u_2, return_sequences=True))
    model.add(LSTM(units=u_3))
    model.add(Dense(units=u_1))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 15- LSTM LSTM LSTM_Densa_DO
# crear_lstm_lstm_lstm_densa_do
def crear_lstm_lstm_lstm_densa_do(u_1,
                                  u_2,
                                  u_3,
                                  pasado_historico,
                                  horizonte_prevision,
                                  dropout=0.1,
                                  n_artributos=1):
    model = Sequential(name="LSTM{}_LSTM{}_LSTM{}_Densa{}_DO{}".format(
        u_1, u_2, u_3, horizonte_prevision,
        int(dropout * 100)))
    model.add(
        LSTM(units=u_1,
             dropout=dropout,
             return_sequences=True,
             input_shape=(pasado_historico, n_artributos)))
    model.add(LSTM(units=u_2, dropout=dropout, return_sequences=True))
    model.add(LSTM(units=u_3, dropout=dropout))
    model.add(Dense(units=u_1))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_lstm_lstm_lstm_densa_do
def crear_masking_lstm_lstm_lstm_densa_do(u_1,
                                          u_2,
                                          u_3,
                                          pasado_historico,
                                          horizonte_prevision,
                                          dropout=0.1,
                                          n_artributos=1):
    model = Sequential(name="Masking_LSTM{}_LSTM{}_LSTM{}_Densa{}_DO{}".format(
        u_1, u_2, u_3, horizonte_prevision,
        int(dropout * 100)))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(LSTM(units=u_1, dropout=dropout, return_sequences=True))
    model.add(LSTM(units=u_2, dropout=dropout, return_sequences=True))
    model.add(LSTM(units=u_3, dropout=dropout))
    model.add(Dense(units=u_1))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 16- GRU Flatten Densa
# crear_gru_flatten_densa
def crear_gru_flatten_densa(u_0,
                            pasado_historico,
                            horizonte_prevision,
                            n_artributos=1):
    model = Sequential(
        name="GRU{}_Flatten_Densa{}".format(u_0, horizonte_prevision))
    model.add(GRU(units=u_0,
                  input_shape=(pasado_historico, n_artributos)))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_gru_flatten_densa
def crear_masking_gru_flatten_densa(u_0,
                                    pasado_historico,
                                    horizonte_prevision,
                                    n_artributos=1):
    model = Sequential(name="Masking_GRU{}_Flatten_Densa{}".format(
        u_0, horizonte_prevision))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(GRU(units=u_0))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 17- GRU DO Flatten Densa
# crear_gru_do_flatten_densa
def crear_gru_do_flatten_densa(u_0,
                               pasado_historico,
                               horizonte_prevision,
                               dropout=0.1,
                               n_artributos=1):
    model = Sequential(name="GRU{}_DO{}_Flatten_Densa{}".format(
        u_0, int(dropout * 100), horizonte_prevision))
    model.add(GRU(units=u_0,
                  input_shape=(pasado_historico, n_artributos)))
    model.add(Dropout(rate=dropout, seed=1))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_gru_do_flatten_densa
def crear_masking_gru_do_flatten_densa(u_0,
                                       pasado_historico,
                                       horizonte_prevision,
                                       dropout=0.1,
                                       n_artributos=1):
    model = Sequential(name="Masking_GRU{}_DO{}_Flatten_Densa{}".format(
        u_0, horizonte_prevision, int(dropout * 100)))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(GRU(units=u_0, dropout=dropout))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 18- GRU GRU Flatten Densa
# crear_gru_gru_flatten_densa
def crear_gru_gru_flatten_densa(u_1,
                                u_2,
                                pasado_historico,
                                horizonte_prevision,
                                n_artributos=1):
    model = Sequential(name="GRU{}_GRU{}_Flatten_Densa{}".format(
        u_1, u_2, horizonte_prevision))
    model.add(
        GRU(units=u_1,
            return_sequences=True,
            input_shape=(pasado_historico, n_artributos)))
    model.add(GRU(units=u_2))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_gru_gru_flatten_densa
def crear_masking_gru_gru_flatten_densa(u_1,
                                        u_2,
                                        pasado_historico,
                                        horizonte_prevision,
                                        n_artributos=1):
    model = Sequential(name="Masking_GRU{}_GRU{}_Flatten_Densa{}".format(
        u_1, u_2, horizonte_prevision))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(GRU(units=u_1, return_sequences=True))
    model.add(GRU(units=u_2))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 19- GRU DO GRU DO Flatten Densa
# crear_gru_do_gru_do_flatten_densa
def crear_gru_do_gru_do_flatten_densa(u_1,
                                      u_2,
                                      pasado_historico,
                                      horizonte_prevision,
                                      dropout=0.1,
                                      n_artributos=1):
    model = Sequential(name="GRU{}_DO{}_GRU{}_DO{}_Flatten_Densa{}".format(
        u_1, int(dropout * 100), 
        u_2, int(dropout * 100), 
        horizonte_prevision))
    model.add(GRU(units=u_1,
                  return_sequences=True,
                  input_shape=(pasado_historico, n_artributos)))
    model.add(Dropout(rate=dropout, seed=1))
    model.add(GRU(units=u_2))
    model.add(Dropout(rate=dropout, seed=1))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_gru_gru_flatten_densa_do
def crear_masking_gru_do_gru_do_flatten_densa(u_1,
                                              u_2,
                                              pasado_historico,
                                              horizonte_prevision,
                                              dropout=0.1,
                                              n_artributos=1):
    model = Sequential(name="Masking_GRU{}_DO{}_GRU{}_DO{}_Flatten_Densa{}".format(
        u_1, int(dropout * 100), 
        u_2, int(dropout * 100), 
        horizonte_prevision))
    model.add(Masking(mask_value=-1, 
                      input_shape=(pasado_historico, n_artributos)))
    model.add(GRU(units=u_1, dropout=dropout, return_sequences=True))
    model.add(GRU(units=u_2, dropout=dropout))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 20- GRU GRU Flatten Densa_Densa
# crear_gru_gru_flatten_densa_densa
def crear_gru_gru_flatten_densa_densa(u_1,
                                      u_2,
                                      pasado_historico,
                                      horizonte_prevision,
                                      n_artributos=1):
    model = Sequential(name="GRU{}_GRU{}_Flatten_Densa{}_Densa{}".format(
        u_1, u_2, u_1, horizonte_prevision))
    model.add(
        GRU(units=u_1,
            return_sequences=True,
            input_shape=(pasado_historico, n_artributos)))
    model.add(GRU(units=u_2))
    model.add(Flatten())
    model.add(Dense(units=u_1))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_gru_gru_flatten_densa_densa
def crear_masking_gru_gru_flatten_densa_densa(u_1,
                                              u_2,
                                              pasado_historico,
                                              horizonte_prevision,
                                              n_artributos=1):
    model = Sequential(
        name="Masking_GRU{}_GRU{}_Flatten_Densa{}_Densa{}".format(
            u_1, u_2, u_1, horizonte_prevision))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(GRU(units=u_1, return_sequences=True))
    model.add(GRU(units=u_2))
    model.add(Flatten())
    model.add(Dense(units=u_1))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 21- GRU GRU Flatten Densa_Densa_DO
# crear_gru_do_gru_do_flatten_densa_densa
def crear_gru_do_gru_do_flatten_densa_densa(u_1,
                                            u_2,
                                            pasado_historico,
                                            horizonte_prevision,
                                            dropout=0.1,
                                            n_artributos=1):
    model = Sequential(
        name="GRU{}_DO{}_GRU{}_DO{}_Flatten_Densa{}_Densa{}".format(
            u_1, int(dropout * 100), u_2, int(dropout * 100),
            u_1, horizonte_prevision))
    model.add(
        GRU(units=u_1,
            return_sequences=True,
            input_shape=(pasado_historico, n_artributos)))
    model.add(Dropout(rate=dropout, seed=1))
    model.add(GRU(units=u_2))
    model.add(Dropout(rate=dropout, seed=1))
    model.add(Flatten())
    model.add(Dense(units=u_1))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_gru_gru_flatten_densa_densa_do
def crear_masking_gru_do_gru_do_flatten_densa_densa(u_1,
                                                    u_2,
                                                    pasado_historico,
                                                    horizonte_prevision,
                                                    dropout=0.1,
                                                    n_artributos=1):
    model = Sequential(
        name="Masking_GRU{}_DO{}_GRU{}_DO{}_Flatten_Densa{}_Densa{}".format(
            u_1, int(dropout * 100), u_2, int(dropout * 100),
            u_1, horizonte_prevision))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(GRU(units=u_1, dropout=dropout, return_sequences=True))
    model.add(GRU(units=u_2, dropout=dropout))
    model.add(Flatten())
    model.add(Dense(units=u_1))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 22- GRU GRU GRU Flatten Densa
# crear_gru_gru_gru_flatten_densa
def crear_gru_gru_gru_flatten_densa(u_1,
                                    u_2,
                                    u_3,
                                    pasado_historico,
                                    horizonte_prevision,
                                    n_artributos=1):
    model = Sequential(name="GRU{}_GRU{}_GRU{}_Flatten_Densa{}".format(
        u_1, u_2, u_3, horizonte_prevision))
    model.add(
        GRU(units=u_1,
            return_sequences=True,
            input_shape=(pasado_historico, n_artributos)))
    model.add(GRU(units=u_2, return_sequences=True))
    model.add(GRU(units=u_3))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_gru_gru_gru_flatten_densa
def crear_masking_gru_gru_gru_flatten_densa(u_1,
                                            u_2,
                                            u_3,
                                            pasado_historico,
                                            horizonte_prevision,
                                            n_artributos=1):
    model = Sequential(name="Masking_GRU{}_GRU{}_GRU{}_Flatten_Densa{}".format(
        u_1, u_2, u_3, horizonte_prevision))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(GRU(units=u_1, return_sequences=True))
    model.add(GRU(units=u_2, return_sequences=True))
    model.add(GRU(units=u_3))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 23- GRU GRU GRU Flatten Densa_DO
# crear_gru_do_gru_do_gru_do_flatten_densa
def crear_gru_do_gru_do_gru_do_flatten_densa(u_1,
                                             u_2,
                                             u_3,
                                             pasado_historico,
                                             horizonte_prevision,
                                             dropout=0.1,
                                             n_artributos=1):
    model = Sequential(
        name="GRU{}_DO{}_GRU{}_DO{}_GRU{}_DO{}_Flatten_Densa{}".format(
            u_1, int(dropout * 100), u_2, int(dropout * 100),
            u_3, int(dropout * 100), horizonte_prevision))
    model.add(
        GRU(units=u_1,
            return_sequences=True,
            input_shape=(pasado_historico, n_artributos)))
    model.add(Dropout(rate=dropout, seed=1))
    model.add(GRU(units=u_2, return_sequences=True))
    model.add(Dropout(rate=dropout, seed=1))
    model.add(GRU(units=u_3))
    model.add(Dropout(rate=dropout, seed=1))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_gru_do_gru_do_gru_do_flatten_densa
def crear_masking_gru_do_gru_do_gru_do_flatten_densa(u_1,
                                                     u_2,
                                                     u_3,
                                                     pasado_historico,
                                                     horizonte_prevision,
                                                     dropout=0.1,
                                                     n_artributos=1):
    model = Sequential(
        name="Masking_GRU{}_DO{}_GRU{}_DO{}_GRU{}_DO{}_Flatten_Densa{}".format(
            u_1, int(dropout * 100), u_2, int(dropout * 100),
            u_3, int(dropout * 100), horizonte_prevision))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(GRU(units=u_1, dropout=dropout, return_sequences=True))
    model.add(GRU(units=u_2, dropout=dropout, return_sequences=True))
    model.add(GRU(units=u_3, dropout=dropout))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 24- Conv1D MaxPool1D Flatten Densa
# crear_conv1d_maxpool1d_flatten_densa
def crear_conv1d_maxpool1d_flatten_densa(u_1,
                                         pasado_historico,
                                         horizonte_prevision,
                                         n_artributos=1):
    model = Sequential(name="Conv1D{}_MaxPool1D{}_Flatten_Densa{}".format(
        u_1, 2, horizonte_prevision))
    model.add(
        Conv1D(filters=u_1,
               kernel_size=7,
               padding='same',
               activation='relu',
               input_shape=(pasado_historico, n_artributos)))
    model.add(MaxPool1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_conv1d_maxpool1d_flatten_densa
def crear_masking_conv1d_maxpool1d_flatten_densa(u_1,
                                                 pasado_historico,
                                                 horizonte_prevision,
                                                 n_artributos=1):
    model = Sequential(
        name="Masking_Conv1D{}_MaxPool1D{}_Flatten_Densa{}".format(
            u_1, 2, horizonte_prevision))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(
        Conv1D(filters=u_1,
               kernel_size=7,
               padding='same',
               activation='relu',
               input_shape=(pasado_historico, n_artributos)))
    model.add(MaxPool1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


#### 25- Conv1D_DO_MaxPool1D_Flatten_Densa
# crear_conv1d_DO_maxpool1d_flatten_densa
def crear_conv1d_do_maxpool1d_flatten_densa(u_1,
                                            pasado_historico,
                                            horizonte_prevision,
                                            dropout=0.1,
                                            n_artributos=1):
    model = Sequential(name="Conv1D{}_DO{}_MaxPool1D{}_Flatten_Densa{}".format(
        u_1, int(dropout * 100), 2, horizonte_prevision))
    model.add(
        Conv1D(filters=u_1,
               kernel_size=7,
               padding='same',
               activation='relu',
               input_shape=(pasado_historico, n_artributos)))
    model.add(Dropout(rate=dropout, seed=1))
    model.add(MaxPool1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_conv1d_DO_maxpool1d_flatten_densa
def crear_masking_conv1d_do_maxpool1d_flatten_densa(u_1,
                                                    pasado_historico,
                                                    horizonte_prevision,
                                                    dropout=0.1,
                                                    n_artributos=1):
    model = Sequential(
        name="Masking_Conv1D{}_DO{}_MaxPool1D{}_Flatten_Densa{}".format(
            u_1, int(dropout * 100), 2, horizonte_prevision))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(
        Conv1D(filters=u_1,
               kernel_size=7,
               padding='same',
               activation='relu',
               input_shape=(pasado_historico, n_artributos)))
    model.add(Dropout(rate=dropout, seed=1))
    model.add(MaxPool1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 26- Conv1D MaxPool1D Conv1D MaxPool1D Flatten Densa
# crear_conv1d_maxpool1d_conv1d_maxpool1d_flatten_densa
def crear_conv1d_maxpool1d_conv1d_maxpool1d_flatten_densa(
        u_1,
        u_2,
        pasado_historico,
        horizonte_prevision,
        n_artributos=1):
    model = Sequential(
        name="Conv1D{}_MaxPool1D{}_Conv1D{}_MaxPool1D{}_Flatten_Densa{}".
        format(u_1, 2, u_2, 2, horizonte_prevision))
    model.add(
        Conv1D(filters=u_1,
               kernel_size=7,
               padding='same',
               activation='relu',
               input_shape=(pasado_historico, n_artributos)))
    model.add(MaxPool1D(pool_size=2))
    model.add(
        Conv1D(filters=u_2,
               kernel_size=5,
               padding='same',
               activation='relu'))
    model.add(MaxPool1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_conv1d_maxpool1d_conv1d_maxpool1d_flatten_densa
def crear_masking_conv1d_maxpool1d_conv1d_maxpool1d_flatten_densa(
        u_1,
        u_2,
        pasado_historico,
        horizonte_prevision,
        n_artributos=1):
    model = Sequential(
        name="Masking_Conv1D{}_MaxPool1D{}_Conv1D{}_MaxPool1D{}_Flatten_Densa{}"
        .format(u_1, 2, u_2, 2, horizonte_prevision))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(
        Conv1D(filters=u_1,
               kernel_size=7,
               padding='same',
               activation='relu'))
    model.add(MaxPool1D(pool_size=2))
    model.add(
        Conv1D(filters=u_2,
               kernel_size=5,
               padding='same',
               activation='relu'))
    model.add(MaxPool1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 27- Conv1D DO MaxPool1D Conv1D DO MaxPool1D Flatten Densa
# crear_conv1d_do_maxpool1d_conv1d_do_maxpool1d_flatten_densa
def crear_conv1d_do_maxpool1d_conv1d_do_maxpool1d_flatten_densa(
        u_1,
        u_2,
        pasado_historico,
        horizonte_prevision,
        dropout=0.1,
        n_artributos=1):
    model = Sequential(
        name=
        "Conv1D{}_DO{}_MaxPool1D{}_Conv1D{}_DO{}_MaxPool1D{}_Flatten_Densa{}".
        format(u_1, int(dropout * 100), 2, u_2, int(
            dropout * 100), 2, horizonte_prevision))
    model.add(
        Conv1D(filters=u_1,
               kernel_size=7,
               padding='same',
               activation='relu',
               input_shape=(pasado_historico, n_artributos)))
    model.add(Dropout(rate=dropout))
    model.add(MaxPool1D(pool_size=2))
    model.add(
        Conv1D(filters=u_2,
               kernel_size=5,
               padding='same',
               activation='relu'))
    model.add(Dropout(rate=dropout))
    model.add(MaxPool1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_conv1d_do_maxpool1d_conv1d_do_maxpool1d_flatten_densa
def crear_masking_conv1d_do_maxpool1d_conv1d_do_maxpool1d_flatten_densa(
        u_1,
        u_2,
        pasado_historico,
        horizonte_prevision,
        dropout=0.1,
        n_artributos=1):
    model = Sequential(
        name=
        "Masking_Conv1D{}_DO{}_MaxPool1D{}_Conv1D{}_DO{}_MaxPool1D{}_Flatten_Densa{}"
        .format(u_1, int(dropout * 100), 2, u_2,
                int(dropout * 100), 2, horizonte_prevision))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(
        Conv1D(filters=u_1,
               kernel_size=7,
               padding='same',
               activation='relu'))
    model.add(Dropout(rate=dropout))
    model.add(MaxPool1D(pool_size=2))
    model.add(
        Conv1D(filters=u_2,
               kernel_size=5,
               padding='same',
               activation='relu'))
    model.add(Dropout(rate=dropout))
    model.add(MaxPool1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


#### 28- TCN_Densa
# crear_TCN_densa
def crear_TCN_densa(u_0,
                    pasado_historico,
                    horizonte_prevision,
                    n_artributos=1):
    model = Sequential(
        name="TCN{}_Densa{}".format(u_0, horizonte_prevision))
    model.add(InputLayer(input_shape=(pasado_historico, n_artributos)))
    model.add(
        TCN(nb_filters=u_0,
            kernel_size=3,
            nb_stacks=1,
            dilations=[1, 2, 4, 8, 16, 32, 64]))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_TCN_densa
def crear_masking_TCN_densa(u_0,
                            pasado_historico,
                            horizonte_prevision,
                            n_artributos=1):
    model = Sequential(
        name="Masking_TCN{}_Densa{}".format(u_0, horizonte_prevision))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(
        TCN(nb_filters=u_0,
            kernel_size=3,
            nb_stacks=1,
            dilations=[1, 2, 4, 8, 16, 32, 64]))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# 29- TCN DO Densa
# crear_TCN_do_densa
def crear_TCN_do_densa(u_0,
                       pasado_historico,
                       horizonte_prevision,
                       dropout=0.1,
                       n_artributos=1):
    model = Sequential(name="TCN{}_DO{}_Densa{}".format(
        u_0, int(dropout * 100), horizonte_prevision))
    model.add(InputLayer(input_shape=(pasado_historico, n_artributos)))
    model.add(
        TCN(nb_filters=u_0,
            kernel_size=3,
            nb_stacks=1,
            dilations=[1, 2, 4, 8, 16, 32, 64]))
    model.add(Dropout(rate=dropout))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model


# crear_masking_TCN_do_densa
def crear_masking_TCN_do_densa(u_0,
                               pasado_historico,
                               horizonte_prevision,
                               dropout=0.1,
                               n_artributos=1):
    model = Sequential(name="Masking_TCN{}_DO{}_Densa{}".format(
        u_0, int(dropout * 100), horizonte_prevision))
    model.add(
        Masking(mask_value=-1, input_shape=(pasado_historico, n_artributos)))
    model.add(
        TCN(nb_filters=u_0,
            kernel_size=3,
            nb_stacks=1,
            dilations=[1, 2, 4, 8, 16, 32, 64]))
    model.add(Dropout(rate=dropout))
    model.add(Dense(units=horizonte_prevision))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    print(model.summary())
    return model

# Directorios

In [ ]:
# Definir el directorio de trabajo
cwd = "/content/drive/MyDrive/Colab Notebooks/DUMMY_NUMBERS_2"
os.chdir(cwd)
print(cwd)

# Crear directorios
data_path = mk_dir("00-data")
plots_path = mk_dir("01-plots")
modelos_path = mk_dir("02-modelos")
tablas_path = mk_dir("03-tablas")
sumario_path = mk_dir("04-sumario")

# Importar datos

In [ ]:
# from google.colab import files
# df = files.upload()

In [ ]:
# Importing dataset
df = pd.read_excel(
    '/content/drive/MyDrive/Colab Notebooks/DUMMY_NUMBERS_2/DEALS_DETAIL-DUMMY_NUMBERS.xlsx')

# Eliminar la columna Negocio - Título
df.drop('Negocio - Título', axis=1, inplace=True)
df.columns = [
    'Creado_el', 'Fecha_ganado', 'Origen', 'CAC', 'Vehículo', 'Contratación',
    'Tipo_cliente', 'Kilometraje', 'Plazo', 'PFF', 'Cuota', 'Comisión_fija',
    'Comision_variable', 'Comsión_total']

df['Vehículo'] = df['Vehículo'].str.replace('Modelo ', '')

# convertir las 'Negocio - Negocio creado el' al formato 'datetime' de pandas
df['Creado_el'] = pd.to_datetime(df['Creado_el'], format='%Y-%m-%d %H:%M:%S')
# convertir las 'Negocio - Fecha de ganado' al formato 'datetime' de pandas
df['Fecha_ganado'] = pd.to_datetime(df['Fecha_ganado'],
                                    format='%Y-%m-%d %H:%M:%S')
# ordenar datos por 'Fecha_creado'
df.sort_values(by=['Creado_el'], inplace=True, ascending=True)

df.reset_index(drop=True, inplace=True)

# Echemos un vistazo al conjunto de datos
df

# CANTIDAD DE VENTAS POR FECHA

In [ ]:
df_fecha_cant = df.resample(
    "D", on='Creado_el').count()['Comsión_total'].to_frame(name="Num_ventas")
plt.plot(df_fecha_cant)
plt.xticks(pd.date_range(df_fecha_cant.index.min(),
                         df_fecha_cant.index.max(),
                         freq='7D'),
           rotation='vertical',
           size=9)
plt.show()

In [ ]:
df_fecha_cant

# COMISIONES POR FECHA

In [ ]:
df_fecha_com = df.resample(
    "D", on='Creado_el').sum()['Comsión_total'].to_frame(name="Comisiones_dia")
plt.plot(df_fecha_com)
plt.xticks(pd.date_range(df_fecha_com.index.min(),
                         df_fecha_com.index.max(),
                         freq='7D'),
           rotation='vertical', size=9)
plt.show()

In [ ]:
df_fecha_com

In [ ]:
df_ = pd.concat([df_fecha_com, df_fecha_cant], axis=1)
df_

# CANTIDAD DE VENTAS POR DÍA DE LA SEMANA

In [ ]:
df_fecha_cant['Día_de_la_semana'] = df_fecha_cant.index.day_name()
df_fecha_cant['Día_de_la_semana'] = df_fecha_cant['Día_de_la_semana'].apply(
    lambda x: traducir_dia(x))

dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']

df_dia_cant = df_fecha_cant.groupby('Día_de_la_semana').sum()['Num_ventas'].to_frame(name="Num_ventas").reindex(dias)

print(df_dia_cant)

plot_bar(df_dia_cant.index, df_dia_cant["Num_ventas"],
         "CANTIDAD DE VENTAS POR DÍA DE LA SEMANA", 
         "DÍA DE LA SEMANA",
         "Número de ventas")

In [ ]:
plot_pie_cant(df_dia_cant.index, df_dia_cant["Num_ventas"])

# COMISIONES POR DÍA DE LA SEMANA

In [ ]:
df_fecha_com['Día_de_la_semana'] = df_fecha_com.index.day_name()
df_fecha_com['Día_de_la_semana'] = df_fecha_com['Día_de_la_semana'].apply(lambda x: traducir_dia(x))
df_dia_com = df_fecha_com.groupby('Día_de_la_semana').sum()['Comisiones_dia'].to_frame(name="Comisiones_dia").reindex(dias)
print(df_dia_com)
plot_bar(df_dia_com.index, df_dia_com["Comisiones_dia"], 
         "COMISIONES TOTALES POR DÍA DE LA SEMANA", 
         "DÍA DE LA SEMANA", 
         "COMISIONES")

In [ ]:
plot_pie_comi(df_dia_com.index, df_dia_com["Comisiones_dia"])


# **Predicciones**


In [ ]:
horizonte_prevision = 7

# Seleccionamos los datos a partir del día 2020-06-17
#df = df_fecha_cant.loc[(df_fecha_cant.index > "2020-06-17"),["Num_ventas"]]
df = df_.loc[(df_.index > "2020-06-01"),]

In [ ]:

for column in tqdm_notebook(df.columns):
    print(column)

# MODELOS TRADICIONALES

## 00- AUTOARIMA

In [ ]:
dir_modelo = "00-AUTOARIMA"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):

    # Regex para filtrar los resultados
    RE = re.compile(r'^AUTOARIMA.+' + column + "$")

    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)

    # datos de entrenamiento y testeo
    df_train = df[[column]].iloc[:-horizonte_prevision].copy()
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    tiempo_inicio_fit = datetime.datetime.now()

    # Nombre de modelo
    nombre_modelo = "AUTOARIMA-" + column
    print("\n\n{}: Modelo \t{}".format(tiempo_inicio_fit, nombre_modelo))

    # Ruta al modelo
    path_modelo = os.path.join(modelo_mercado_path, nombre_modelo + ".pkl")
    if os.path.isfile(path_modelo):  # Si el modelo existe
        # Cargar modelo
        print("\n{}: Cargando modelo:\t{}".format(datetime.datetime.now(),path_modelo))
        AUTOARIMA = pickle.load(open(path_modelo, 'rb'))
    else:
        # Crear modelo
        print("\n{}: Inicio entrenamiento".format(datetime.datetime.now()))
        AUTOARIMA = auto_arima(df_train[column].values)
        # Guardar modelo
        print("\n{}: Guardando modelo:\t{}".format(datetime.datetime.now(),path_modelo))
        pickle.dump(AUTOARIMA, open(path_modelo, 'wb'))
        # Tiempo de entrenamiento
        tiempo_entrenamiento()

    # Sumario
    print("\n{}: SUMARIO:\n{}".format(datetime.datetime.now(), AUTOARIMA.summary()))

    # Diagnósticos
    diagnostic(AUTOARIMA)

    # Parámetros
    print("\n{}: Parámetros:\n{}".format(datetime.datetime.now(), AUTOARIMA.get_params()))

    # Pronóstico
    fc = prediction(AUTOARIMA)

    # Plot pronóstico
    plot_pronos(df_test, nombre_modelo, column)

    # Métricas
    metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)

    gc.collect()
    
    # Plot Métricas
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

## 01- AR


In [ ]:
dir_modelo = "01-AR"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    # Regex para filtrar los resultados
    RE = re.compile(r'^AR.+' + column + "$")
    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    df_train = df[[column]].dropna().iloc[:-horizonte_prevision].copy()
    df_test = df[[column]].dropna().iloc[-horizonte_prevision:].copy()
    for p in tqdm_notebook(range(1, 3)):
        tiempo_inicio_fit = datetime.datetime.now()

        # Nombre de modelo
        nombre_modelo = 'AR_p{:02d}-{}'.format(p, column)
        print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                           nombre_modelo))

        # Ruta al modelo
        path_modelo = os.path.join(modelo_mercado_path, nombre_modelo + ".pkl")
        if os.path.isfile(path_modelo):  # Si el modelo existe

            # Cargar modelo
            print("\n{}: Cargando modelo:\t{}".format(datetime.datetime.now(),
                                                      path_modelo))
            AR_result = pickle.load(open(path_modelo, 'rb'))
        else:  # Si el modelo no existe
            # Crear modelo
            AR = ARMA(df_train[column], order=(p, 0))

            # Entrenar modelo
            print("\n{}: Inicio entrenamiento".format(datetime.datetime.now()))
            AR_result = AR.fit()

            # Guardar modelo
            print("\n{}: Guardando modelo:\t{}".format(datetime.datetime.now(),
                                                       path_modelo))
            AR_result.save(path_modelo)
            pickle.dump(AR_result, open(path_modelo, 'wb'))

            # Tiempo de entrenamiento
            tiempo_entrenamiento()

        # Sumario
        print("\n{}: Sumario:\n{}".format(datetime.datetime.now(), AR_result.summary()))

        # Plot predicción
        AR_result.plot_predict()
        plt.show()

        # Pronóstico
        fc = forecast(AR_result)

        # Plot pronóstico
        plot_pronos(df_test, nombre_modelo, column)

        # Métricas
        metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)

        gc.collect()
    # Plot Métricas
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

## 02- MA

In [ ]:
dir_modelo = "02-MA"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    # Regex para filtrar los resultados
    RE = re.compile(r'^MA.+' + column + "$")
    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    df_train = df[[column]].dropna().iloc[:-horizonte_prevision].copy()
    df_test = df[[column]].dropna().iloc[-horizonte_prevision:].copy()
    for q in tqdm_notebook(range(1, 4)):
        try:
            tiempo_inicio_fit = datetime.datetime.now()
            # Nombre de modelo
            nombre_modelo = 'MA_q{:02d}-{}'.format(q, column)
            # Ruta al modelo
            path_modelo = os.path.join(modelo_mercado_path,
                                       nombre_modelo + ".pkl")
            if os.path.isfile(path_modelo):  # Si el modelo existe
                # Cargar modelo
                print("\n{}: Cargando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                MA_result = pickle.load(open(path_modelo, 'rb'))
            else:  # Si el modelo no existe
                # Crear modelo
                MA = ARMA(df_train[column], order=(0, q))
                # Entrenar modelo
                print("\n{}: Inicio entrenamiento".format(
                    datetime.datetime.now()))
                MA_result = MA.fit()
                # Guardar modelo
                print("\n{}: Guardando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                MA_result.save(path_modelo)
                pickle.dump(MA_result, open(path_modelo, 'wb'))
                # Tiempo de entrenamiento
                tiempo_entrenamiento()

            # Sumario
            print("\n{}: Sumario:\n{}".format(datetime.datetime.now(), MA_result.summary()))

            # Pronóstico
            fc = forecast(MA_result)

            # Plot predicción
            MA_result.plot_predict(start=pd.to_datetime("2020-09-01"))
            plt.show()

            # Plot pronóstico
            plot_pronos(df_test, nombre_modelo, column)

            # Métricas
            metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)

            gc.collect()
        except:
            continue
    
    # Plot Métricas
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

## 03- ARMA

In [ ]:
p = q = range(1, 3)
pq = list(itertools.product(p, q))
dir_modelo = "03-ARMA"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    # Regex para filtrar los resultados
    RE = re.compile(r'^ARMA.+' + column + "$")
    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    df_train = df[[column]].dropna().iloc[:-horizonte_prevision].copy()
    df_test = df[[column]].dropna().iloc[-horizonte_prevision:].copy()
    for parameters in tqdm_notebook(pq):
        try:
            tiempo_inicio_fit = datetime.datetime.now()
            # Nombre de modelo
            nombre_modelo = 'ARMA_p{:02d}q{:02d}-{}'.format(
                parameters[0], parameters[1], column)
            print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                               nombre_modelo))
            # Ruta al modelo
            path_modelo = os.path.join(modelo_mercado_path,
                                       nombre_modelo + ".pkl")

            if os.path.isfile(path_modelo):  # Si el modelo existe
                # Cargar modelo
                print("\n{}: Cargando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                ARMA_result = ARIMAResults.load(path_modelo)
            else:  # Si el modelo no existe
                # Crear modelo
                ARMA_ = ARMA(df_train[column], order=parameters)
                print("\n{}: Inicio entrenamiento".format(
                    datetime.datetime.now()))
                # Entrenar modelo
                ARMA_result = ARMA_.fit()
                # Guardar modelo
                print("\n{}: Guardando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                ARMA_result.save(path_modelo)

                # Tiempo de entrenamiento
                tiempo_entrenamiento()

            # Sumario
            print("\n{}: Sumario:\n{}".format(datetime.datetime.now(), ARMA_result.summary()))

            # Pronóstico
            fc = forecast(ARMA_result)

            # Plot pronóstico
            plot_pronos(df_test, nombre_modelo, column)

            # Métricas
            metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)

            gc.collect()
        except:
            continue
    # Plot Métricas
    try:
        plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)
    except:
        continue    

## 04- ARIMA



In [ ]:
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))
dir_modelo = "04-ARIMA"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    # Regex para filtrar los resultados
    RE = re.compile(r'^ARIMA.+' + column + "$")
    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    df_train = df[[column]].dropna().iloc[:-horizonte_prevision].copy()
    df_test = df[[column]].dropna().iloc[-horizonte_prevision:].copy()
    for parameters in pdq:
        try:
            tiempo_inicio_fit = datetime.datetime.now()
            # Nombre de modelo
            nombre_modelo = 'ARIMA_p{:02d}d{:02d}q{:02d}-{}'.format(
                parameters[0], parameters[1], parameters[2], column)
            print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                               nombre_modelo))
            # Ruta al modelo
            path_modelo = os.path.join(modelo_mercado_path,
                                       nombre_modelo + ".pkl")
            if os.path.isfile(path_modelo):  # Si el modelo existe
                # Cargar modelo
                print("\n{}: Cargando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                ARIMA_result = ARIMAResults.load(path_modelo)
            else:  # Si el modelo no existe
                # Crear modelo
                ARIMA_ = ARIMA(df_train[column], order=parameters)
                print("\n{}: Inicio entrenamiento".format(
                    datetime.datetime.now()))
                # Entrenar modelo
                ARIMA_result = ARIMA_.fit()
                # Guardar modelo
                print("\n{}: Guardando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                ARIMA_result.save(path_modelo)
                # Tiempo de entrenamiento
                tiempo_entrenamiento()
            # Sumario
            print("\n{}: Sumario:\n{}".format(datetime.datetime.now(),ARIMA_result.summary()))

            # Pronóstico
            fc = forecast(ARIMA_result)
    
            # Plot pronóstico
            plot_pronos(df_test, nombre_modelo, column)
            
            # Métricas
            metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)

            gc.collect()
        except:
            continue
    # Plot Métricas
    try:
        # Plot Métricas
        plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)
    except:
        continue

## 05- SARIMA

In [ ]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq_comb = [(i[0], i[1], i[2], 7)
                     for i in list(itertools.product(p, d, q))]

dir_modelo = "05-SARIMA"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    # Regex para filtrar los resultados
    RE = re.compile(r'^SARIMA.+' + column + "$")
    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    df_train = df[[column]].dropna().iloc[:-horizonte_prevision].copy()
    df_test = df[[column]].dropna().iloc[-horizonte_prevision:].copy()
    for parameters in tqdm_notebook(pdq):
        for seasonal_param in tqdm_notebook(seasonal_pdq_comb):
            try:
                tiempo_inicio_fit = datetime.datetime.now()
                # Nombre de modelo
                nombre_modelo = 'SARIMA_p{:02d}d{:02d}q{:02d}P{:02d}D{:02d}Q{:02d}S{:02d}-{}'.format(
                    parameters[0], parameters[1], parameters[2],
                    seasonal_param[0], seasonal_param[1], seasonal_param[2],
                    seasonal_param[3], column)
                print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                                   nombre_modelo))
                # Ruta al modelo
                path_modelo = os.path.join(modelo_mercado_path,
                                           nombre_modelo + ".pkl")
                # Si el modelo existe
                if os.path.isfile(path_modelo):
                    # Cargar modelo
                    print("\n{}: Cargando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    SARIMA_result = SARIMAXResults.load(path_modelo)
                else:  # Si el modelo no existe
                    # Crear modelo
                    SARIMA = SARIMAX(df_train[column],
                                     order=parameters,
                                     seasonal_param_order=seasonal_param,
                                     enforce_stationarity=False,
                                     enforce_invertibility=False)
                    print("\n{}: Inicio entrenamiento".format(
                        datetime.datetime.now()))
                    # Entrenar modelo
                    SARIMA_result = SARIMA.fit()
                    # Guardar modelo
                    print("\n{}: Guardando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    SARIMA_result.save(path_modelo)
                    # Tiempo de entrenamiento
                    tiempo_entrenamiento()

                # Sumario
                print("\n{}: Sumario:\n{}".format(datetime.datetime.now(), SARIMA_result.summary()))

                # Pronóstico
                fc = forecast(SARIMA_result)
                
                # Plot pronóstico
                plot_pronos(df_test, nombre_modelo, column)
                
                # Predicción
                plot_pronos_sarima(SARIMA_result, df_test[["Num_ventas"]])
                
                # Métricas
                metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
                gc.collect()
            except:
                continue

    try:
        # Plot Métricas
        plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)
    except:
        continue

## 06- BATS

In [ ]:
dir_modelo = "06-BATS"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):

    # Regex para filtrar los resultados
    RE = re.compile(r'^BATS.+' + column +
                    "$")
    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    df_train = df[[column]].dropna().iloc[:-horizonte_prevision].copy()
    df_test = df[[column]].dropna().iloc[-horizonte_prevision:].copy()
    periodo_1 = 7
    periodo_2 = 2 * 7
    try:
        tiempo_inicio_fit = datetime.datetime.now()
        # Nombre de modelo
        nombre_modelo = "BATS_"+'%i' % periodo_1+"_"+'%i' % periodo_2 + \
            "-"+column
        print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                           nombre_modelo))
        # Ruta al modelo
        path_modelo = os.path.join(modelo_mercado_path, nombre_modelo + ".pkl")

        if os.path.isfile(path_modelo):  # Si el modelo existe
            # Cargar modelo
            print("\n{}: Cargando modelo:\t{}".format(datetime.datetime.now(),
                                                      path_modelo))
            BATS_result = pickle.load(open(path_modelo, 'rb'))
        else:  # Si el modelo no existe
            # Crear modelo
            BATS_ = BATS(seasonal_periods=(periodo_1, periodo_2))
            print("\n{}: Inicio entrenamiento".format(datetime.datetime.now()))
            # Entrenar modelo
            BATS_result = BATS_.fit(df_train[column])
            # Guardar modelo
            print("\n{}: Guardando modelo:\t{}".format(datetime.datetime.now(),
                                                       path_modelo))
            pickle.dump(BATS_result, open(path_modelo, 'wb'))

            # Tiempo de entrenamiento
            tiempo_entrenamiento()

        # Pronóstico
        forecast_bats(BATS_result)
        
        # Plot pronóstico
        plot_pronos(df_test, nombre_modelo, column)

        # Métricas
        metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)

        gc.collect()
    except:
        continue
    # Plot Métricas
    try:
        plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)
    except:
        continue

## 07- TBATS

In [ ]:
dir_modelo = "07-TBATS"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):

    # Regex para filtrar los resultados
    RE = re.compile(r'^TBATS.+' + column +
                    "$")
    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    df_train = df[[column]].dropna().iloc[:-horizonte_prevision].copy()
    df_test = df[[column]].dropna().iloc[-horizonte_prevision:].copy()
    periodo_1 = 7
    periodo_2 = 2 * 7
    try:
        tiempo_inicio_fit = datetime.datetime.now()
        # Nombre de modelo
        nombre_modelo = "TBATS_"+'%i' % periodo_1+"_"+'%i' % periodo_2 + \
            "-"+column
        print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                           nombre_modelo))
        # Ruta al modelo
        path_modelo = os.path.join(modelo_mercado_path, nombre_modelo + ".pkl")

        if os.path.isfile(path_modelo):  # Si el modelo existe
            # Cargar modelo
            print("\n{}: Cargando modelo:\t{}".format(datetime.datetime.now(),
                                                      path_modelo))
            TBATS_result = pickle.load(open(path_modelo, 'rb'))
        else:  # Si el modelo no existe
            # Crear modelo
            TBATS_ = TBATS(seasonal_periods=(periodo_1, periodo_2))
            print("\n{}: Inicio entrenamiento".format(datetime.datetime.now()))
            # Entrenar modelo
            TBATS_result = TBATS_.fit(df_train[column])
            # Guardar modelo
            print("\n{}: Guardando modelo:\t{}".format(datetime.datetime.now(),
                                                       path_modelo))
            pickle.dump(TBATS_result, open(path_modelo, 'wb'))

            # Tiempo de entrenamiento
            tiempo_entrenamiento()

        # Pronóstico
        forecast_bats(TBATS_result)

        # Plot pronóstico
        plot_pronos(df_test, nombre_modelo, column)

        # Métricas
        metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)

        gc.collect()
    except:
        continue
    # Plot Métricas
    try:
        plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)
    except:
        continue

# REDES NEURONALES RECURRENTES

In [ ]:
pasado_historico = 7*4
scaler = MinMaxScaler(feature_range=(0, 1))
mis_callbacks = [EarlyStopping(monitor='val_loss', patience=3, mode='min', restore_best_weights=True)]

## Long short-term memory 

### 08- LSTM Densa

In [ ]:
dir_modelo = "08-LSTM_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_0 in tqdm_notebook([32, 64]):
        for batch_size in tqdm_notebook([128]):
            tiempo_inicio_fit = datetime.datetime.now()
            match = re.search("[3-7]", column)
            if not match:
                RE = re.compile(r'^LSTM\d+?_Densa\d+?.+' +
                                column +
                                '$')
                # with tpu_strategy.scope():
                model = crear_lstm_densa(u_0,
                                         pasado_historico,
                                         horizonte_prevision,
                                         n_artributos=1)
            else:
                RE = re.compile(r'^Masking_LSTM\d+?_Densa\d+?.+' +
                                column +
                                '$')
                # with tpu_strategy.scope():
                model = crear_masking_lstm_densa(u_0,
                                                 pasado_historico,
                                                 horizonte_prevision,
                                                 n_artributos=1)
            # Nombre de modelo
            nombre_modelo = '{}_batch{}-{}'.format(
                model.name, batch_size,
                column)
            print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                               nombre_modelo))
            # Ruta al modelo
            path_modelo = os.path.join(modelo_mercado_path,
                                       nombre_modelo + ".h5")
            if os.path.isfile(path_modelo):
                # Cargar modelo
                print("\n{}: Cargando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                model = keras.models.load_model(path_modelo)
            else:
                # Entrenar modelo
                history = fit_model(x_train, y_train, batch_size)
                
                # Tiempo de entrenamiento
                tiempo_entrenamiento()

                # Guardar modelo
                print("\n{}: Guardando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                model.save(path_modelo)
                
                # Plot history
                plot_history(history)

            # Info del modelo
            print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
            pprint.pprint(model.get_config())            

            # Pronóstico
            fc = forecast_nn()

            # Plot pronóstico
            plot_pronos(df_test, nombre_modelo, column)

            # Métricas
            metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
            gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 09- LSTM Densa DO

In [ ]:
dir_modelo = "09-LSTM_Densa_DO"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_0 in tqdm_notebook([32, 64]):
        for batch_size in tqdm_notebook([128]):
            tiempo_inicio_fit = datetime.datetime.now()
            match = re.search("[3-7]", column)
            if not match:
                RE = re.compile(r'^LSTM\d+?_Densa\d+?_DO.+' +
                                column +
                                '$')
                # with tpu_strategy.scope():
                model = crear_lstm_densa_do(u_0,
                                            pasado_historico,
                                            horizonte_prevision,
                                            dropout=0.1,
                                            n_artributos=1)
            else:
                RE = re.compile(r'^Masking_LSTM\d+?_Densa\d+?_DO.+' +
                                column +
                                '$')
                # with tpu_strategy.scope():
                model = crear_masking_lstm_densa_do(u_0,
                                                    pasado_historico,
                                                    horizonte_prevision,
                                                    dropout=0.1,
                                                    n_artributos=1)
            # Nombre de modelo
            nombre_modelo = '{}_batch{}-{}'.format(
                model.name, batch_size,
                column)
            print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                               nombre_modelo))
            # Ruta al modelo
            path_modelo = os.path.join(modelo_mercado_path,
                                       nombre_modelo + ".h5")
            if os.path.isfile(path_modelo):
                # Cargar modelo
                print("\n{}: Cargando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                model = keras.models.load_model(path_modelo)
            else:
                # Entrenar modelo
                history = fit_model(x_train, y_train, batch_size)
                
                # Tiempo de entrenamiento
                tiempo_entrenamiento()

                # Guardar modelo
                print("\n{}: Guardando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                model.save(path_modelo)
                
                # Plot history
                plot_history(history)

            # Info del modelo
            print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
            pprint.pprint(model.get_config())

            # Pronóstico
            fc = forecast_nn()

            # Plot pronóstico
            plot_pronos(df_test, nombre_modelo, column)

            # Métricas
            metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)

            gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 10- LSTM LSTM Densa

In [ ]:
dir_modelo = "10-LSTM_LSTM_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_1 in tqdm_notebook([32, 64]):
        for u_2 in tqdm_notebook([32, 64]):
            for batch_size in tqdm_notebook([128]):
                tiempo_inicio_fit = datetime.datetime.now()
                match = re.search("[3-7]", column)
                if not match:
                    RE = re.compile(
                        r'^LSTM\d+?_LSTM\d+?_Densa\d+?.+' +
                        column + '$')
                    # with tpu_strategy.scope():
                    model = crear_lstm_lstm_densa(u_1,
                                                  u_2,
                                                  pasado_historico,
                                                  horizonte_prevision,
                                                  n_artributos=1)
                else:
                    RE = re.compile(
                        r'^Masking_LSTM\d+?_LSTM\d+?_Densa\d+?.+' +
                        column + '$')
                    # with tpu_strategy.scope():
                    model = crear_masking_lstm_lstm_densa(u_1,
                                                          u_2,
                                                          pasado_historico,
                                                          horizonte_prevision,
                                                          n_artributos=1)
                # Nombre de modelo
                nombre_modelo = '{}_batch{}-{}'.format(
                    model.name, batch_size,
                    column)
                print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                                   nombre_modelo))
                # Ruta al modelo
                path_modelo = os.path.join(modelo_mercado_path,
                                           nombre_modelo + ".h5")
                if os.path.isfile(path_modelo):
                    # Cargar modelo
                    print("\n{}: Cargando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model = keras.models.load_model(path_modelo)
                else:
                    # Entrenar modelo
                    history = fit_model(x_train, y_train, batch_size)
                    
                    # Tiempo de entrenamiento
                    tiempo_entrenamiento()

                    # Guardar modelo
                    print("\n{}: Guardando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model.save(path_modelo)
                    
                    # Plot history
                    plot_history(history)

                # Info del modelo
                print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
                pprint.pprint(model.get_config())

                # Pronóstico
                fc = forecast_nn()

                # Plot pronóstico
                plot_pronos(df_test, nombre_modelo, column)

                # Métricas
                metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
                gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 11- LSTM LSTM Densa DO

In [ ]:
dir_modelo = "11-LSTM_LSTM_Densa_DO"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_1 in tqdm_notebook([32, 64]):
        for u_2 in tqdm_notebook([32, 64]):
            for batch_size in tqdm_notebook([128]):
                tiempo_inicio_fit = datetime.datetime.now()
                match = re.search("[3-7]", column)
                if not match:
                    RE = re.compile(
                        r'^LSTM\d+?_LSTM\d+?_Densa\d+?_DO.+' +
                        column + '$')
                    # with tpu_strategy.scope():
                    model = crear_lstm_lstm_densa_do(u_1,
                                                     u_2,
                                                     pasado_historico,
                                                     horizonte_prevision,
                                                     dropout=0.1,
                                                     n_artributos=1)
                else:
                    RE = re.compile(
                        r'^Masking_LSTM\d+?_LSTM\d+?_Densa\d+?_DO.+' +
                        column + '$')
                    # with tpu_strategy.scope():
                    model = crear_masking_lstm_lstm_densa_do(
                        u_1,
                        u_2,
                        pasado_historico,
                        horizonte_prevision,
                        dropout=0.1,
                        n_artributos=1)
                # Nombre de modelo
                nombre_modelo = '{}_batch{}-{}'.format(
                    model.name, batch_size,
                    column)
                print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                                   nombre_modelo))
                # Ruta al modelo
                path_modelo = os.path.join(modelo_mercado_path,
                                           nombre_modelo + ".h5")
                if os.path.isfile(path_modelo):
                    # Cargar modelo
                    print("\n{}: Cargando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model = keras.models.load_model(path_modelo)
                else:
                    # Entrenar modelo
                    history = fit_model(x_train, y_train, batch_size)
                    
                    # Tiempo de entrenamiento
                    tiempo_entrenamiento()

                    # Guardar modelo
                    print("\n{}: Guardando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model.save(path_modelo)
                    
                    # Plot history
                    plot_history(history)

                # Info del modelo               
                print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
                pprint.pprint(model.get_config())

                # Pronóstico
                fc = forecast_nn()

                # Plot pronóstico
                plot_pronos(df_test, nombre_modelo, column)

                # Métricas
                metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
                gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 12- LSTM LSTM Densa Densa

In [ ]:
dir_modelo = "12-LSTM_LSTM_Densa_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_1 in tqdm_notebook([32, 64]):
        for u_2 in tqdm_notebook([32, 64]):
            for batch_size in tqdm_notebook([128]):
                tiempo_inicio_fit = datetime.datetime.now()
                match = re.search("[3-7]", column)
                if not match:
                    RE = re.compile(
                        r'^LSTM\d+?_LSTM\d+?_Densa\d+?_Densa\d+?.+' +
                        column + '$')
                    # with tpu_strategy.scope():
                    model = crear_lstm_lstm_densa_densa(u_1,
                                                        u_2,
                                                        pasado_historico,
                                                        horizonte_prevision,
                                                        n_artributos=1)
                else:
                    RE = re.compile(
                        r'^Masking_LSTM\d+?_LSTM\d+?_Densa\d+?_Densa\d+?.+' +
                        column + '$')
                    # with tpu_strategy.scope():
                    model = crear_masking_lstm_lstm_densa_densa(
                        u_1,
                        u_2,
                        pasado_historico,
                        horizonte_prevision,
                        n_artributos=1)
                # Nombre de modelo
                nombre_modelo = '{}_batch{}-{}'.format(
                    model.name, batch_size,
                    column)
                print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                                   nombre_modelo))
                # Ruta al modelo
                path_modelo = os.path.join(modelo_mercado_path,
                                           nombre_modelo + ".h5")
                if os.path.isfile(path_modelo):
                    # Cargar modelo
                    print("\n{}: Cargando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model = keras.models.load_model(path_modelo)
                else:
                    # Entrenar modelo
                    history = fit_model(x_train, y_train, batch_size)
                    
                    # Tiempo de entrenamiento
                    tiempo_entrenamiento()

                    # Guardar modelo
                    print("\n{}: Guardando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model.save(path_modelo)
                    
                    # Plot history
                    plot_history(history)

                # Info del modelo               
                print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
                pprint.pprint(model.get_config())

                # Pronóstico
                fc = forecast_nn()

                # Plot pronóstico
                plot_pronos(df_test, nombre_modelo, column)

                # Métricas
                metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
                gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 13- LSTM LSTM Densa Densa DO

In [ ]:
dir_modelo = "13-LSTM_LSTM_Densa_Densa_DO"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_1 in tqdm_notebook([32, 64]):
        for u_2 in tqdm_notebook([32, 64]):
            for batch_size in tqdm_notebook([128]):
                tiempo_inicio_fit = datetime.datetime.now()
                match = re.search("[3-7]", column)
                if not match:
                    RE = re.compile(
                        r'^LSTM\d+?_LSTM\d+?_Densa\d+?_Densa\d+?_DO.+' +
                        column + '$')
                    # with tpu_strategy.scope():
                    model = crear_lstm_lstm_densa_densa_do(u_1,
                                                           u_2,
                                                           pasado_historico,
                                                           horizonte_prevision,
                                                           dropout=0.1,
                                                           n_artributos=1)
                else:
                    RE = re.compile(
                        r'^Masking_LSTM\d+?_LSTM\d+?_Densa\d+?_Densa\d+?_DO.+'
                        + column + '$')
                    # with tpu_strategy.scope():
                    model = crear_masking_lstm_lstm_densa_densa_do(
                        u_1,
                        u_2,
                        pasado_historico,
                        horizonte_prevision,
                        dropout=0.1,
                        n_artributos=1)
                # Nombre de modelo
                nombre_modelo = '{}_batch{}-{}'.format(
                    model.name, batch_size,
                    column)
                print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                                   nombre_modelo))
                # Ruta al modelo
                path_modelo = os.path.join(modelo_mercado_path,
                                           nombre_modelo + ".h5")
                if os.path.isfile(path_modelo):
                    # Cargar modelo
                    print("\n{}: Cargando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model = keras.models.load_model(path_modelo)
                else:
                    # Entrenar modelo
                    history = fit_model(x_train, y_train, batch_size)
                    
                    # Tiempo de entrenamiento
                    tiempo_entrenamiento()

                    # Guardar modelo
                    print("\n{}: Guardando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model.save(path_modelo)
                    
                    # Plot history
                    plot_history(history)

                # Info del modelo              
                print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
                pprint.pprint(model.get_config())

                # Pronóstico
                fc = forecast_nn()

                # Plot pronóstico
                plot_pronos(df_test, nombre_modelo, column)

                # Métricas
                metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
                gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 14- LSTM LSTM LSTM Densa

In [ ]:
dir_modelo = "14-LSTM_LSTM_LSTM_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_1 in tqdm_notebook([32, 64]):
        for u_2 in tqdm_notebook([32, 64]):
            for u_3 in tqdm_notebook([32, 64]):
                for batch_size in tqdm_notebook([128]):
                    tiempo_inicio_fit = datetime.datetime.now()
                    match = re.search("[3-7]", column)
                    if not match:
                        RE = re.compile(
                            r'^LSTM\d+?_LSTM\d+?_LSTM\d+?_Densa\d+?.+' +
                            column + '$')
                        # with tpu_strategy.scope():
                        model = crear_lstm_lstm_lstm_densa(u_1,
                                                           u_2,
                                                           u_3,
                                                           pasado_historico,
                                                           horizonte_prevision,
                                                           n_artributos=1)
                    else:
                        RE = re.compile(
                            r'^Masking_LSTM\d+?_LSTM\d+?_LSTM\d+?_Densa\d+?.+'
                            + column + '$')
                        # with tpu_strategy.scope():
                        model = crear_masking_lstm_lstm_lstm_densa(
                            u_1,
                            u_2,
                            u_3,
                            pasado_historico,
                            horizonte_prevision,
                            n_artributos=1)
                    # Nombre de modelo
                    nombre_modelo = '{}_batch{}-{}'.format(
                        model.name, batch_size,
                        column)
                    print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                                       nombre_modelo))
                    # Ruta al modelo
                    path_modelo = os.path.join(modelo_mercado_path,
                                               nombre_modelo + ".h5")
                    if os.path.isfile(path_modelo):
                        # Cargar modelo
                        print("\n{}: Cargando modelo:\t{}".format(
                            datetime.datetime.now(), path_modelo))
                        model = keras.models.load_model(path_modelo)
                    else:
                        # Entrenar modelo
                        history = fit_model(x_train, y_train, batch_size)
                        
                        # Tiempo de entrenamiento
                        tiempo_entrenamiento()

                        # Guardar modelo
                        print("\n{}: Guardando modelo:\t{}".format(
                            datetime.datetime.now(), path_modelo))
                        model.save(path_modelo)
                        
                        # Plot history
                        plot_history(history)

                    # Info del modelo                   
                    print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
                    pprint.pprint(model.get_config())

                    # Pronóstico
                    fc = forecast_nn()

                    # Plot pronóstico
                    plot_pronos(df_test, nombre_modelo, column)

                    # Métricas
                    metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
                    gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 15- LSTM LSTM LSTM Densa DO

In [ ]:
dir_modelo = "15-LSTM_LSTM_LSTM_Densa_DO"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_1 in tqdm_notebook([32, 64]):
        for u_2 in tqdm_notebook([32, 64]):
            for u_3 in tqdm_notebook([32, 64]):
                for batch_size in tqdm_notebook([128]):
                    tiempo_inicio_fit = datetime.datetime.now()
                    match = re.search("[3-7]", column)
                    if not match:
                        RE = re.compile(
                            r'^LSTM\d+?_LSTM\d+?_LSTM\d+?_Densa\d+?_DO.+' +
                            column + '$')
                        # with tpu_strategy.scope():
                        model = crear_lstm_lstm_lstm_densa_do(
                            u_1,
                            u_2,
                            u_3,
                            pasado_historico,
                            horizonte_prevision,
                            dropout=0.1,
                            n_artributos=1)
                    else:
                        RE = re.compile(
                            r'^Masking_LSTM\d+?_LSTM\d+?_LSTM\d+?_Densa\d+?_DO.+'
                            + column + '$')
                        # with tpu_strategy.scope():
                        model = crear_masking_lstm_lstm_lstm_densa_do(
                            u_1,
                            u_2,
                            u_3,
                            pasado_historico,
                            horizonte_prevision,
                            dropout=0.1,
                            n_artributos=1)
                    # Nombre de modelo
                    nombre_modelo = '{}_batch{}-{}'.format(
                        model.name, batch_size,
                        column)
                    print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                                       nombre_modelo))
                    # Ruta al modelo
                    path_modelo = os.path.join(modelo_mercado_path,
                                               nombre_modelo + ".h5")
                    if os.path.isfile(path_modelo):
                        # Cargar modelo
                        print("\n{}: Cargando modelo:\t{}".format(
                            datetime.datetime.now(), path_modelo))
                        model = keras.models.load_model(path_modelo)
                    else:
                        # Entrenar modelo
                        history = fit_model(x_train, y_train, batch_size)
                        
                        # Tiempo de entrenamiento
                        tiempo_entrenamiento()

                        # Guardar modelo
                        print("\n{}: Guardando modelo:\t{}".format(
                            datetime.datetime.now(), path_modelo))
                        model.save(path_modelo)
                        
                        # Plot history
                        plot_history(history)

                    # Info del modelo                    
                    print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
                    pprint.pprint(model.get_config())

                    # Pronóstico
                    fc = forecast_nn()

                    # Plot pronóstico
                    plot_pronos(df_test, nombre_modelo, column)

                    # Métricas
                    metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
                    gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

## Gated recurrent unit

### 16- GRU Flatten Densa

In [ ]:
dir_modelo = "16-GRU_Flatten_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_0 in tqdm_notebook([32, 64]):
        for batch_size in tqdm_notebook([128]):
            tiempo_inicio_fit = datetime.datetime.now()
            match = re.search("[3-7]", column)
            if not match:
                RE = re.compile(r'^GRU\d+?_Flatten_Densa\d+?.+' +
                                column +
                                '$')
                # with tpu_strategy.scope():
                model = crear_gru_flatten_densa(u_0,
                                                pasado_historico,
                                                horizonte_prevision,
                                                n_artributos=1)
            else:
                RE = re.compile(r'^Masking_GRU\d+?_Flatten_Densa\d+?.+' +
                                column +
                                '$')
                # with tpu_strategy.scope():
                model = crear_masking_gru_flatten_densa(u_0,
                                                        pasado_historico,
                                                        horizonte_prevision,
                                                        n_artributos=1)
            # Nombre de modelo
            nombre_modelo = '{}_batch{}-{}'.format(
                model.name, batch_size,
                column)
            print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                               nombre_modelo))
            # Ruta al modelo
            path_modelo = os.path.join(modelo_mercado_path,
                                       nombre_modelo + ".h5")
            if os.path.isfile(path_modelo):
                # Cargar modelo
                print("\n{}: Cargando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                model = keras.models.load_model(path_modelo)
            else:
                # Entrenar modelo
                history = fit_model(x_train, y_train, batch_size)
                
                # Tiempo de entrenamiento
                tiempo_entrenamiento()

                # Guardar modelo
                print("\n{}: Guardando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                model.save(path_modelo)
                
                # Plot history
                plot_history(history)

            # Info del modelo            
            print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
            pprint.pprint(model.get_config())

            # Pronóstico
            fc = forecast_nn()

            # Plot pronóstico
            plot_pronos(df_test, nombre_modelo, column)

            # Métricas
            metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
            gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 17- GRU DO Flatten Densa 

In [ ]:
dir_modelo = "17-GRU_DO_Flatten_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_0 in tqdm_notebook([32, 64]):
        for batch_size in tqdm_notebook([128]):
            tiempo_inicio_fit = datetime.datetime.now()
            match = re.search("[3-7]", column)
            if not match:
                RE = re.compile(r'^GRU\d+?_DO\d+?_Flatten_Densa\d+?.+' +
                                column +
                                '$')
                # with tpu_strategy.scope():
                model = crear_gru_do_flatten_densa(u_0,
                                                   pasado_historico,
                                                   horizonte_prevision,
                                                   dropout=0.1,
                                                   n_artributos=1)
            else:
                RE = re.compile(
                    r'^Masking_GRU\d+?_DO\d+?_Flatten_Densa\d+?.+' +
                    column + '$')
                # with tpu_strategy.scope():
                model = crear_masking_gru_flatten_densa_do(u_0,
                                                           pasado_historico,
                                                           horizonte_prevision,
                                                           dropout=0.1,
                                                           n_artributos=1)
            # Nombre de modelo
            nombre_modelo = '{}_batch{}-{}'.format(
                model.name, batch_size,
                column)
            print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                               nombre_modelo))
            # Ruta al modelo
            path_modelo = os.path.join(modelo_mercado_path,
                                       nombre_modelo + ".h5")
            if os.path.isfile(path_modelo):
                # Cargar modelo
                print("\n{}: Cargando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                model = keras.models.load_model(path_modelo)
                print("\n{}: Sumario: ".format(datetime.datetime.now()))
                pprint.pprint(model.summary())
            else:
                # Entrenar modelo
                history = fit_model(x_train, y_train, batch_size)
                
                # Tiempo de entrenamiento
                tiempo_entrenamiento()

                # Guardar modelo
                print("\n{}: Guardando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                model.save(path_modelo)
                
                # Plot history
                plot_history(history)

            # Info del modelo            
            print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
            pprint.pprint(model.get_config())

            # Pronóstico
            fc = forecast_nn()

            # Plot pronóstico
            plot_pronos(df_test, nombre_modelo, column)

            # Métricas
            metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
            gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 18- GRU GRU Flatten Densa

In [ ]:
dir_modelo = "18-GRU_GRU_Flatten_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_1 in tqdm_notebook([32, 64]):
        for u_2 in tqdm_notebook([32, 64]):
            for batch_size in tqdm_notebook([128]):
                tiempo_inicio_fit = datetime.datetime.now()
                match = re.search("[3-7]", column)
                if not match:
                    RE = re.compile(
                        r'^GRU\d+?_GRU\d+?_Flatten_Densa\d+?.+' +
                        column + '$')
                    # with tpu_strategy.scope():
                    model = crear_gru_gru_flatten_densa(u_1,
                                                        u_2,
                                                        pasado_historico,
                                                        horizonte_prevision,
                                                        n_artributos=1)
                else:
                    RE = re.compile(
                        r'^Masking_GRU\d+?_GRU\d+?_Flatten_Densa\d+?.+' +
                        column + '$')
                    # with tpu_strategy.scope():
                    model = crear_masking_gru_gru_flatten_densa(
                        u_1,
                        u_2,
                        pasado_historico,
                        horizonte_prevision,
                        n_artributos=1)
                # Nombre de modelo
                nombre_modelo = '{}_batch{}-{}'.format(
                    model.name, batch_size,
                    column)
                print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                                   nombre_modelo))
                # Ruta al modelo
                path_modelo = os.path.join(modelo_mercado_path,
                                           nombre_modelo + ".h5")
                if os.path.isfile(path_modelo):
                    # Cargar modelo
                    print("\n{}: Cargando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model = keras.models.load_model(path_modelo)
                else:
                    # Entrenar modelo
                    history = fit_model(x_train, y_train, batch_size)
                    
                    # Tiempo de entrenamiento
                    tiempo_entrenamiento()

                    # Guardar modelo
                    print("\n{}: Guardando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model.save(path_modelo)
                    
                    # Plot history
                    plot_history(history)

                # Info del modelo                
                print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
                pprint.pprint(model.get_config())

                # Pronóstico
                fc = forecast_nn()

                # Plot pronóstico
                plot_pronos(df_test, nombre_modelo, column)

                # Métricas
                metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
                gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 19- GRU DO GRU DO Flatten Densa

In [ ]:
dir_modelo = "19-GRU_DO_GRU_DO_Flatten_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):

    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_1 in tqdm_notebook([32, 64]):
        for u_2 in tqdm_notebook([32, 64]):
            for batch_size in tqdm_notebook([128]):
                tiempo_inicio_fit = datetime.datetime.now()
                match = re.search("[3-7]", column)
                if not match:
                    RE = re.compile(
                        r'^GRU\d+?_DO\d+?_GRU\d+?_DO\d+?_Flatten_Densa\d+?.+' +
                        column + '$')
                    # with tpu_strategy.scope():
                    model = crear_gru_do_gru_do_flatten_densa(
                        u_1,
                        u_2,
                        pasado_historico,
                        horizonte_prevision,
                        dropout=0.1,
                        n_artributos=1)
                else:
                    RE = re.compile(
                        r'^Masking_GRU\d+?_DO\d+?_GRU\d+?_DO\d+?_Flatten_Densa\d+?.+'
                        + column + '$')
                    # with tpu_strategy.scope():
                    model = crear_masking_gru_do_gru_do_flatten_densa(
                        u_1,
                        u_2,
                        pasado_historico,
                        horizonte_prevision,
                        dropout=0.1,
                        n_artributos=1)
                # Nombre de modelo
                nombre_modelo = '{}_batch{}-{}'.format(
                    model.name, batch_size,
                    column)
                print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                                   nombre_modelo))
                # Ruta al modelo
                path_modelo = os.path.join(modelo_mercado_path,
                                           nombre_modelo + ".h5")
                if os.path.isfile(path_modelo):
                    # Cargar modelo
                    print("\n{}: Cargando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model = keras.models.load_model(path_modelo)
                else:
                    # Entrenar modelo
                    history = fit_model(x_train, y_train, batch_size)
                    
                    # Tiempo de entrenamiento
                    tiempo_entrenamiento()

                    # Guardar modelo
                    print("\n{}: Guardando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model.save(path_modelo)
                    
                    # Plot history
                    plot_history(history)

                # Info del modelo                
                print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
                pprint.pprint(model.get_config())

                # Pronóstico
                fc = forecast_nn()

                # Plot pronóstico
                plot_pronos(df_test, nombre_modelo, column)

                # Métricas
                metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
                gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 20- GRU GRU Flatten Densa Densa

In [ ]:
dir_modelo = "20-GRU_GRU_Flatten_Densa_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):

    # Regex para filtrar los resultados

    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_1 in tqdm_notebook([32, 64]):
        for u_2 in tqdm_notebook([32, 64]):
            for batch_size in tqdm_notebook([128]):
                tiempo_inicio_fit = datetime.datetime.now()
                match = re.search("[3-7]", column)
                if not match:
                    RE = re.compile(
                        r'^GRU\d+?_GRU\d+?_Flatten_Densa\d+?_Densa\d+?.+' +
                        column + '$')
                    # with tpu_strategy.scope():
                    model = crear_gru_gru_flatten_densa_densa(
                        u_1,
                        u_2,
                        pasado_historico,
                        horizonte_prevision,
                        n_artributos=1)
                else:
                    RE = re.compile(
                        r'^Masking_GRU\d+?_GRU\d+?_Flatten_Densa\d+?_Densa\d+?.+'
                        + column + '$')
                    # with tpu_strategy.scope():
                    model = crear_masking_gru_gru_flatten_densa_densa(
                        u_1,
                        u_2,
                        pasado_historico,
                        horizonte_prevision,
                        n_artributos=1)
                # Nombre de modelo
                nombre_modelo = '{}_batch{}-{}'.format(
                    model.name, batch_size,
                    column)
                print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                                   nombre_modelo))
                # Ruta al modelo
                path_modelo = os.path.join(modelo_mercado_path,
                                           nombre_modelo + ".h5")
                if os.path.isfile(path_modelo):
                    # Cargar modelo
                    print("\n{}: Cargando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model = keras.models.load_model(path_modelo)
                else:
                    # Entrenar modelo
                    history = fit_model(x_train, y_train, batch_size)
                    
                    # Tiempo de entrenamiento
                    tiempo_entrenamiento()

                    # Guardar modelo
                    print("\n{}: Guardando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model.save(path_modelo)
                    
                    # Plot history
                    plot_history(history)

                # Info del modelo                
                print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
                pprint.pprint(model.get_config())

                # Pronóstico
                fc = forecast_nn()

                # Plot pronóstico
                plot_pronos(df_test, nombre_modelo, column)

                # Métricas
                metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
                gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 21- GRU DO GRU DO Flatten Densa Densa

In [ ]:
dir_modelo = "21-GRU_DO_GRU_DO_Flatten_Densa_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):

    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_1 in tqdm_notebook([32, 64]):
        for u_2 in tqdm_notebook([32, 64]):
            for batch_size in tqdm_notebook([128]):
                tiempo_inicio_fit = datetime.datetime.now()
                match = re.search("[3-7]", column)
                if not match:
                    RE = re.compile(
                        r'^GRU\d+?_DO\d+?_GRU\d+?_DO\d+?_Flatten_Densa\d+?_Densa\d+?.+'
                        + column + '$')
                    # with tpu_strategy.scope():
                    model = crear_gru_do_gru_do_flatten_densa_densa(
                        u_1,
                        u_2,
                        pasado_historico,
                        horizonte_prevision,
                        dropout=0.1,
                        n_artributos=1)
                else:
                    RE = re.compile(
                        r'^Masking_GRU\d+?_DO\d+?_GRU\d+?_DO\d+?_Flatten_Densa\d+?_Densa\d+?.+'
                        + column + '$')
                    # with tpu_strategy.scope():
                    model = crear_masking_gru_do_gru_do_flatten_densa_densa(
                        u_1,
                        u_2,
                        pasado_historico,
                        horizonte_prevision,
                        dropout=0.1,
                        n_artributos=1)
                # Nombre de modelo
                nombre_modelo = '{}_batch{}-{}'.format(
                    model.name, batch_size,
                    column)
                print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                                   nombre_modelo))
                # Ruta al modelo
                path_modelo = os.path.join(modelo_mercado_path,
                                           nombre_modelo + ".h5")
                if os.path.isfile(path_modelo):
                    # Cargar modelo
                    print("\n{}: Cargando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model = keras.models.load_model(path_modelo)
                else:
                    # Entrenar modelo
                    history = fit_model(x_train, y_train, batch_size)
                    
                    # Tiempo de entrenamiento
                    tiempo_entrenamiento()

                    # Guardar modelo
                    print("\n{}: Guardando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model.save(path_modelo)
                    
                    # Plot history
                    plot_history(history)

                # Info del modelo                
                print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
                pprint.pprint(model.get_config())

                # Pronóstico
                fc = forecast_nn()

                # Plot pronóstico
                plot_pronos(df_test, nombre_modelo, column)

                # Métricas
                metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
                gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 22- GRU GRU GRU Flatten Densa  

In [ ]:
dir_modelo = "22-GRU_GRU_GRU_Flatten_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    # Regex para filtrar los resultados

    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_1 in tqdm_notebook([32, 64]):
        for u_2 in tqdm_notebook([32, 64]):
            for u_3 in tqdm_notebook([32, 64]):
                for batch_size in tqdm_notebook([128]):
                    tiempo_inicio_fit = datetime.datetime.now()
                    match = re.search("[3-7]", column)
                    if not match:
                        RE = re.compile(
                            r'^GRU\d+?_GRU\d+?_GRU\d+?_Flatten_Densa\d+?.+' +
                            column + '$')
                        # with tpu_strategy.scope():
                        model = crear_gru_gru_gru_flatten_densa(
                            u_1,
                            u_2,
                            u_3,
                            pasado_historico,
                            horizonte_prevision,
                            n_artributos=1)
                    else:
                        RE = re.compile(
                            r'^Masking_GRU\d+?_GRU\d+?_GRU\d+?_Flatten_Densa\d+?.+'
                            + column + '$')
                        # with tpu_strategy.scope():
                        model = crear_masking_gru_gru_gru_flatten_densa(
                            u_1,
                            u_2,
                            u_3,
                            pasado_historico,
                            horizonte_prevision,
                            n_artributos=1)
                    # Nombre de modelo
                    nombre_modelo = '{}_batch{}-{}'.format(
                        model.name, batch_size,
                        column)
                    print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                                       nombre_modelo))
                    # Ruta al modelo
                    path_modelo = os.path.join(modelo_mercado_path,
                                               nombre_modelo + ".h5")
                    if os.path.isfile(path_modelo):
                        # Cargar modelo
                        print("\n{}: Cargando modelo:\t{}".format(
                            datetime.datetime.now(), path_modelo))
                        model = keras.models.load_model(path_modelo)
                    else:
                        # Entrenar modelo
                        history = fit_model(x_train, y_train, batch_size)
                        
                        # Tiempo de entrenamiento
                        tiempo_entrenamiento()

                        # Guardar modelo
                        print("\n{}: Guardando modelo:\t{}".format(
                            datetime.datetime.now(), path_modelo))
                        model.save(path_modelo)
                        
                        # Plot history
                        plot_history(history)

                    # Info del modelo                    
                    print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
                    pprint.pprint(model.get_config())

                    # Pronóstico
                    fc = forecast_nn()

                    # Plot pronóstico
                    plot_pronos(df_test, nombre_modelo, column)

                    # Métricas
                    metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
                    gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 23- GRU DO GRU DO GRU DO Flatten Densa

In [ ]:
dir_modelo = "23-GRU_DO_GRU_DO_GRU_DO_Flatten_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):

    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_1 in tqdm_notebook([32, 64]):
        for u_2 in tqdm_notebook([32, 64]):
            for u_3 in tqdm_notebook([32, 64]):
                for batch_size in tqdm_notebook([128]):
                    tiempo_inicio_fit = datetime.datetime.now()
                    match = re.search("[3-7]", column)
                    if not match:
                        RE = re.compile(
                            r'^GRU\d+?_DO\d+?_GRU\d+?_DO\d+?_GRU\d+?_DO\d+?_Flatten_Densa\d+?.+'
                            + column + '$')
                        # with tpu_strategy.scope():
                        model = crear_gru_do_gru_do_gru_do_flatten_densa(
                            u_1,
                            u_2,
                            u_3,
                            pasado_historico,
                            horizonte_prevision,
                            dropout=0.1,
                            n_artributos=1)
                    else:
                        RE = re.compile(
                            r'^Masking_GRU\d+?_DO\d+?_GRU\d+?_DO\d+?_GRU\d+?_DO\d+?_Flatten_Densa\d+?.+'
                            + column + '$')
                        # with tpu_strategy.scope():
                        model = crear_masking_gru_do_gru_do_gru_do_flatten_densa(
                            u_1,
                            u_2,
                            u_3,
                            pasado_historico,
                            horizonte_prevision,
                            dropout=0.1,
                            n_artributos=1)
                    # Nombre de modelo
                    nombre_modelo = '{}_batch{}-{}'.format(
                        model.name, batch_size,
                        column)
                    print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                                       nombre_modelo))
                    # Ruta al modelo
                    path_modelo = os.path.join(modelo_mercado_path,
                                               nombre_modelo + ".h5")
                    if os.path.isfile(path_modelo):
                        # Cargar modelo
                        print("\n{}: Cargando modelo:\t{}".format(
                            datetime.datetime.now(), path_modelo))
                        model = keras.models.load_model(path_modelo)
                    else:
                        # Entrenar modelo
                        history = fit_model(x_train, y_train, batch_size)
                        
                        # Tiempo de entrenamiento
                        tiempo_entrenamiento()

                        # Guardar modelo
                        print("\n{}: Guardando modelo:\t{}".format(
                            datetime.datetime.now(), path_modelo))
                        model.save(path_modelo)
                        
                        # Plot history
                        plot_history(history)

                    # Info del modelo                    
                    print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
                    pprint.pprint(model.get_config())

                    # Pronóstico
                    fc = forecast_nn()

                    # Plot pronóstico
                    plot_pronos(df_test, nombre_modelo, column)

                    # Métricas
                    metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
                    gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

# REDES NEURONALES CONVOLUCIONALES

## Convolutional Networks

### 24- Conv1D MaxPool1D Flatten Densa

In [ ]:
dir_modelo = "24-Conv1D_MaxPool1D_Flatten_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_0 in tqdm_notebook([32, 64]):
        for batch_size in tqdm_notebook([128]):
            tiempo_inicio_fit = datetime.datetime.now()
            match = re.search("[3-7]", column)
            if not match:
                RE = re.compile(r'^Conv1D\d+?_MaxPool1D2_Flatten_Densa\d+?.+' +
                                column +
                                '$')
                # with tpu_strategy.scope():
                model = crear_conv1d_maxpool1d_flatten_densa(
                    u_0,
                    pasado_historico,
                    horizonte_prevision,
                    n_artributos=1)
            else:
                RE = re.compile(
                    r'^Masking_Conv1D\d+?_MaxPool1D2_Flatten_Densa\d+?.+' +
                    column + '$')
                # with tpu_strategy.scope():
                model = crear_masking_conv1d_maxpool1d_flatten_densa(
                    u_1,
                    pasado_historico,
                    horizonte_prevision,
                    n_artributos=1)
            # Nombre de modelo
            nombre_modelo = '{}_batch{}-{}'.format(
                model.name, batch_size,
                column)
            print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                               nombre_modelo))
            # Ruta al modelo
            path_modelo = os.path.join(modelo_mercado_path,
                                       nombre_modelo + ".h5")
            if os.path.isfile(path_modelo):
                # Cargar modelo
                print("\n{}: Cargando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                model = keras.models.load_model(path_modelo)
            else:
                # Entrenar modelo
                history = fit_model(x_train, y_train, batch_size)
                
                # Tiempo de entrenamiento
                tiempo_entrenamiento()

                # Guardar modelo
                print("\n{}: Guardando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                model.save(path_modelo)
                
                # Plot history
                plot_history(history)

            # Info del modelo            
            print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
            pprint.pprint(model.get_config())

            # Pronóstico
            fc = forecast_nn()

            # Plot pronóstico
            plot_pronos(df_test, nombre_modelo, column)

            # Métricas
            metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
            gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 25- Conv1D DO MaxPool1D Flatten Densa

In [ ]:
dir_modelo = "25-Conv1D_do_MaxPool1D_Flatten_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_0 in tqdm_notebook([32, 64]):
        for batch_size in tqdm_notebook([128]):
            tiempo_inicio_fit = datetime.datetime.now()
            match = re.search("[3-7]", column)
            if not match:
                RE = re.compile(
                    r'^Conv1D\d+?_DO\d+?_MaxPool1D2_Flatten_Densa\d+?.+' +
                    column + '$')
                # with tpu_strategy.scope():
                model = crear_conv1d_do_maxpool1d_flatten_densa(
                    u_0,
                    pasado_historico,
                    horizonte_prevision,
                    dropout=0.1,
                    n_artributos=1)
            else:
                RE = re.compile(
                    r'^Masking_Conv1D\d+?_DO\d+?_MaxPool1D2_Flatten_Densa\d+?.+'
                    + column + '$')
                # with tpu_strategy.scope():
                model = crear_masking_conv1d_do_maxpool1d_flatten_densa(
                    u_1,
                    pasado_historico,
                    horizonte_prevision,
                    dropout=0.1,
                    n_artributos=1)
            # Nombre de modelo
            nombre_modelo = '{}_batch{}-{}'.format(
                model.name, batch_size,
                column)
            print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                               nombre_modelo))
            # Ruta al modelo
            path_modelo = os.path.join(modelo_mercado_path,
                                       nombre_modelo + ".h5")
            if os.path.isfile(path_modelo):
                # Cargar modelo
                print("\n{}: Cargando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                model = keras.models.load_model(path_modelo)
            else:
                # Entrenar modelo
                history = fit_model(x_train, y_train, batch_size)
                
                # Tiempo de entrenamiento
                tiempo_entrenamiento()

                # Guardar modelo
                print("\n{}: Guardando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                model.save(path_modelo)
                
                # Plot history
                plot_history(history)

            # Info del modelo            
            print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
            pprint.pprint(model.get_config())

            # Pronóstico
            fc = forecast_nn()

            # Plot pronóstico
            plot_pronos(df_test, nombre_modelo, column)

            # Métricas
            metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
            gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 26- Conv1D MaxPool1D Conv1D MaxPool1D Flatten Densa

In [ ]:
dir_modelo = "26-Conv1D_MaxPool1D_Conv1D_MaxPool1D_Flatten_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_1 in tqdm_notebook([32, 64]):
        for u_2 in tqdm_notebook([32, 64]):
            for batch_size in tqdm_notebook([128]):
                tiempo_inicio_fit = datetime.datetime.now()
                match = re.search("[3-7]", column)
                if not match:
                    RE = re.compile(
                        r'^Conv1D\d+?_MaxPool1D2_Conv1D\d+?_MaxPool1D2_Flatten_Densa\d+?.+'
                        + column + '$')
                    # with tpu_strategy.scope():
                    model = crear_conv1d_maxpool1d_conv1d_maxpool1d_flatten_densa(
                        u_1,
                        u_2,
                        pasado_historico,
                        horizonte_prevision,
                        n_artributos=1)
                else:
                    RE = re.compile(
                        r'^Masking_Conv1D\d+?_MaxPool1D2_Conv1D\d+?_MaxPool1D2_Flatten_Densa\d+?.+'
                        + column + '$')
                    # with tpu_strategy.scope():
                    model = crear_masking_conv1d_maxpool1d_conv1d_maxpool1d_flatten_densa(
                        u_1,
                        u_2,
                        pasado_historico,
                        horizonte_prevision,
                        n_artributos=1)
                # Nombre de modelo
                nombre_modelo = '{}_batch{}-{}'.format(
                    model.name, batch_size,
                    column)
                print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                                   nombre_modelo))
                # Ruta al modelo
                path_modelo = os.path.join(modelo_mercado_path,
                                           nombre_modelo + ".h5")
                if os.path.isfile(path_modelo):
                    # Cargar modelo
                    print("\n{}: Cargando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model = keras.models.load_model(path_modelo)
                else:
                    # Entrenar modelo
                    history = fit_model(x_train, y_train, batch_size)
                    
                    # Tiempo de entrenamiento
                    tiempo_entrenamiento()

                    # Guardar modelo
                    print("\n{}: Guardando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model.save(path_modelo)
                    
                    # Plot history
                    plot_history(history)

                # Info del modelo                
                print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
                pprint.pprint(model.get_config())

                # Pronóstico
                fc = forecast_nn()

                # Plot pronóstico
                plot_pronos(df_test, nombre_modelo, column)

                # Métricas
                metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
                gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)


### 27- Conv1D DO MaxPool1D Conv1D DO MaxPool1D Flatten Densa

In [ ]:
dir_modelo = "27-Conv1D_DO_MaxPool1D_Conv1D_DO_MaxPool1D_Flatten_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):
    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_1 in tqdm_notebook([32, 64]):
        for u_2 in tqdm_notebook([32, 64]):
            for batch_size in tqdm_notebook([128]):
                tiempo_inicio_fit = datetime.datetime.now()
                match = re.search("[3-7]", column)
                if not match:
                    RE = re.compile(
                        r'^Conv1D\d+?_DO\d+?_MaxPool1D2_Conv1D\d+?_DO\d+?_MaxPool1D2_Flatten_Densa\d+?.+'
                        + column + '$')
                    # with tpu_strategy.scope():
                    model = crear_conv1d_do_maxpool1d_conv1d_do_maxpool1d_flatten_densa(
                        u_1,
                        u_2,
                        pasado_historico,
                        horizonte_prevision,
                        dropout=0.1,
                        n_artributos=1)
                else:
                    RE = re.compile(
                        r'^Masking_Conv1D\d+?_DO\d+?_MaxPool1D2_Conv1D\d+?_DO\d+?_MaxPool1D2_Flatten_Densa\d+?.+'
                        + column + '$')
                    # with tpu_strategy.scope():
                    model = crear_masking_conv1d_do_maxpool1d_conv1d_do_maxpool1d_flatten_densa(
                        u_1,
                        u_2,
                        pasado_historico,
                        horizonte_prevision,
                        dropout=0.1,
                        n_artributos=1)
                # Nombre de modelo
                nombre_modelo = '{}_batch{}-{}'.format(
                    model.name, batch_size,
                    column)
                print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                                   nombre_modelo))
                # Ruta al modelo
                path_modelo = os.path.join(modelo_mercado_path,
                                           nombre_modelo + ".h5")
                if os.path.isfile(path_modelo):
                    # Cargar modelo
                    print("\n{}: Cargando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model = keras.models.load_model(path_modelo)
                else:
                    # Entrenar modelo
                    history = fit_model(x_train, y_train, batch_size)
                    
                    # Tiempo de entrenamiento
                    tiempo_entrenamiento()

                    # Guardar modelo
                    print("\n{}: Guardando modelo:\t{}".format(
                        datetime.datetime.now(), path_modelo))
                    model.save(path_modelo)
                    
                    # Plot history
                    plot_history(history)

                # Info del modelo                
                print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
                pprint.pprint(model.get_config())

                # Pronóstico
                fc = forecast_nn()

                # Plot pronóstico
                plot_pronos(df_test, nombre_modelo, column)

                # Métricas
                metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
                gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

## Temporal Convolutional Networks

### 28- TCN Densa

In [ ]:
dir_modelo = "28-TCN_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):

    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_0 in tqdm_notebook([32, 64]):
        for batch_size in tqdm_notebook([128]):
            tiempo_inicio_fit = datetime.datetime.now()
            match = re.search("[3-7]", column)
            if not match:
                RE = re.compile(r'^TCN\d+?_Densa\d+?.+' +
                                column +
                                '$')
                #with tpu_strategy.scope():
                model = crear_TCN_densa(u_0,
                                        pasado_historico,
                                        horizonte_prevision,
                                        n_artributos=1)
            else:
                RE = re.compile(r'^Masking_TCN\d+?_Densa\d+?.+' +
                                column +
                                '$')
                #with tpu_strategy.scope():
                model = crear_masking_TCN_densa(u_0,
                                                pasado_historico,
                                                horizonte_prevision,
                                                n_artributos=1)
            # Nombre de modelo
            nombre_modelo = '{}_batch{}-{}'.format(
                model.name, batch_size,
                column)
            print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                               nombre_modelo))
            # Ruta al modelo
            path_modelo = os.path.join(modelo_mercado_path,
                                       nombre_modelo + ".tf")
            if os.path.isdir(path_modelo):
                # Cargar modelo
                print("\n{}: Cargando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                model = keras.models.load_model(path_modelo)
            else:
                # Entrenar modelo
                history = fit_model(x_train, y_train, batch_size)
                
                # Tiempo de entrenamiento
                tiempo_entrenamiento()

                # Guardar modelo
                print("\n{}: Guardando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                model.save(path_modelo)
                
                # Plot history
                plot_history(history)

            # Info del modelo            
            print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
            pprint.pprint(model.get_config())

            # Pronóstico
            fc = forecast_nn()

            # Plot pronóstico
            plot_pronos(df_test, nombre_modelo, column)

            # Métricas
            metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
            gc.collect()
    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

### 29- TCN DO Densa

In [ ]:
dir_modelo = "29-TCN_DO_Densa"
modelo_path, tabla_path, plot_path = mk_model_dirs(dir_modelo)
for column in tqdm_notebook(df.columns):

    # Directorios de modelo y mercado/s
    modelo_mercado_path, tabla_mercado_path, plot_mercado_path = mk_model_mercado_dirs(
        column)
    # datos de entrenamiento y testeo
    
    dataset = scaler.fit_transform(df[[column]])
    test = dataset[-horizonte_prevision:]
    train = dataset[:-horizonte_prevision]
    x_train, y_train, x_test, y_test = datos_supervisados(
        train, test, pasado_historico)
    df_test = df[[column]].iloc[-horizonte_prevision:].copy()
    for u_0 in tqdm_notebook([32, 64]):
        for batch_size in tqdm_notebook([128]):
            tiempo_inicio_fit = datetime.datetime.now()
            match = re.search("[3-7]", column)
            if not match:
                RE = re.compile(r'^TCN\d+?_DO\d+?_Densa\d+?.+' +
                                column +
                                '$')
                # with tpu_strategy.scope():
                model = crear_TCN_do_densa(u_0,
                                           pasado_historico,
                                           horizonte_prevision,
                                           dropout=0.1,
                                           n_artributos=1)
            else:
                RE = re.compile(r'^Masking_TCN\d+?_DO\d+?_Densa\d+?.+' +
                                column +
                                '$')
                # with tpu_strategy.scope():
                model = crear_masking_TCN_do_densa(u_0,
                                                   pasado_historico,
                                                   horizonte_prevision,
                                                   dropout=0.1,
                                                   n_artributos=1)
            # Nombre de modelo
            nombre_modelo = '{}_batch{}-{}'.format(
                model.name, batch_size,
                column)
            print("\n\n{}: Modelo \t{}".format(datetime.datetime.now(),
                                               nombre_modelo))
            # Ruta al modelo
            path_modelo = os.path.join(modelo_mercado_path,
                                       nombre_modelo + ".tf")
            if os.path.isdir(path_modelo):
                # Cargar modelo
                print("\n{}: Cargando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                model = keras.models.load_model(path_modelo)
            else:
                # Entrenar modelo
                history = fit_model(x_train, y_train, batch_size)
                
                # Tiempo de entrenamiento
                tiempo_entrenamiento()

                # Guardar modelo
                print("\n{}: Guardando modelo:\t{}".format(
                    datetime.datetime.now(), path_modelo))
                model.save(path_modelo)
                
                # Plot history
                plot_history(history)

            # Info del modelo            
            print("\n{}: Configuración del modelo:".format(datetime.datetime.now()))
            pprint.pprint(model.get_config())

            # Pronóstico
            fc = forecast_nn()

            # Plot pronóstico
            plot_pronos(df_test, nombre_modelo, column)

            # Métricas
            metrics_time_series(df_test[column], fc, res_pronos, nombre_modelo)
            gc.collect()

    plot_metrics(pd.DataFrame(res_pronos), plot_mercado_path, RE)

# RESULTADOS

## Recolectar métricas

In [ ]:
path_metr = os.path.join(sumario_path, "00-metricas_sumario.csv")
if os.path.isfile(path_metr):
    metricas_df = pd.read_csv(path_metr)
    metricas_df.set_index('Modelo', inplace=True)
else:
    metricas_lst = glob.glob(tablas_path + '/*/*/*_m*.csv')
    metrica_data = []
    for filename in tqdm_notebook(metricas_lst):
        data = pd.read_csv(filename)
        metrica_data.append(data)
    metricas_data_concat = pd.concat(metrica_data)
    metricas_data_concat.rename(columns={"Unnamed: 0": "Modelo"}, inplace=True)
    metricas_data_concat.replace(regex=r'-Precio_mercado_SPOT_',
                                 value='-',
                                 inplace=True)
    metricas_data_concat.sort_values(
        by=["R2", "EVS", "MAE", "MSE", "MAPE", "ME"],
        ascending=[False, False, True, True, True, True],
        inplace=True)
    metricas_data_concat.drop_duplicates(keep='first', inplace=True)
    metricas_data_concat.set_index('Modelo', inplace=True)
    metricas_df = metricas_data_concat[~metricas_data_concat.index.duplicated(
        keep='first')]
    metricas_df.to_csv(path_metr)

## Mejor modelo

In [ ]:
for column in tqdm_notebook(df.columns):
    RE = re.compile(r'.*?' + column + "$")
    mejor_modelo = metricas_df.filter(regex=RE, axis=0).head(1)
    mejor_modelo_path = os.path.join(
        sumario_path, "01-mejor_modelo-" + column + "-metricas_sumario.csv")
    if not mejor_modelo.empty:
        mejor_modelo.to_csv(mejor_modelo_path)
        print(mejor_modelo)

## Coeficiente de determinación

In [ ]:
metrica = "R2"
for column in tqdm_notebook(df.columns):
    re_AUTOARIMA = re.compile(r'^AUTOARIMA_100.+' +
                              column + "$")
    met = metricas_df.filter(regex=column,
                             axis=0)
    met_AUTOARIMA = met.filter(regex=re_AUTOARIMA, axis=0)
    met = met.sort_values(by=[metrica], ascending=False).head(20)
    met.sort_values(by=["R2", "EVS", "MAE", "MSE", "MAPE", "ME"],
                    ascending=[False, False, True, True, True, True],
                    inplace=True)
    met = met.head(20)
    met = pd.concat([met, met_AUTOARIMA], axis=0)
    path_met = os.path.join(
        sumario_path, "01-" + metrica + "-" + column + "-metricas_sumario.csv")
    print(path_met)
    print(met)
    met.to_csv(path_met)
    met.sort_values(by=[metrica], ascending=True, inplace=True)
    plt.rcParams['figure.figsize'] = (15, int(len(met) / 1.5))
    met[metrica].plot(kind='barh',
                      fontsize=20,
                      color=tuple(np.where(met[metrica] < 0, 'r', 'b')))
    for index, value in enumerate(met[metrica]):
        plt.text(value,
                 index,
                 str(round(value, 3)),
                 size=18,
                 fontweight="bold")
    plt.title(column.replace("_", " ") + '\nCoeficiente de determinación (R2)',
              size=28)
    plt.ylabel('Modelos', fontsize=26)
    path_met_plot = os.path.join(
        sumario_path, "02-" + metrica + "-" + column + "-metricas_sumario.png")
    print(path_met_plot)
    plt.savefig(path_met_plot, bbox_inches="tight")
    plt.grid(True)
    plt.show()

## Recolectar pronósticos

In [ ]:
path_prons = os.path.join(sumario_path, "03-pronosticos_sumario.csv")
if os.path.isfile(path_prons):
    pronos_df = pd.read_csv(path_prons)
    pronos_df.set_index('Creado_el', inplace=True)
else:
    pronosticos_data = []
    pronosticos_lst = glob.glob(tablas_path + '/*/*/*_p.csv')
    for filename in tqdm_notebook(pronosticos_lst):
        data = pd.read_csv(filename)
        pronosticos_data.append(data)

    pronos_data = []
    for df_ in tqdm_notebook(pronosticos_data):
        if len(df_) == 7:
            df_ = df_.set_index('Creado_el')
            pronos_data.append(df_)

    pronos_df = pd.concat(pronos_data, axis=1)
    pronos_df.drop_duplicates(inplace=True)
    pronos_df = pronos_df.T.drop_duplicates().T
    pronos_df = pronos_df.rename(
        columns=lambda s: s.replace('-Precio_mercado_SPOT_', '-'))
    pronos_df = pronos_df.loc[:, ~pronos_df.columns.duplicated()]
    pronos_df.to_csv(path_prons)

## Plot "top" por modelo y mercado

In [ ]:
top = 1  # top 1
for column in tqdm_notebook(df.columns):
    regexs = mk_regexs(column)
    for reg in tqdm_notebook(regexs):
        mejor_modelo = metricas_df.filter(regex=reg, axis=0).head(top)
        if not mejor_modelo.empty:
            print(mejor_modelo)
            mejores_modelos = mejor_modelo.index.to_list()
            mejores_modelos.insert(0, column)
            pronos_df.filter(items=mejores_modelos).plot(figsize=(15, 7))
            plt.ylabel('Ventas')
            plt.xlabel('Tiempo')
            plt.title("Mejor pronósticos para:\n" + column.replace("_", " "),
                      size=18)
            plt.legend(loc=2)
            path_mejor_plot = os.path.join(
                sumario_path,
                "05-" + column + "-" +
                mejores_modelos[1] + "-pronosticos_sumario.png")
            plt.grid(True)
            plt.savefig(path_mejor_plot, bbox_inches="tight")
            plt.show()